In [ ]:
# Importing functions and classes we'll use

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Dropout, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from warnings import catch_warnings
from warnings import filterwarnings
from tqdm import tqdm
import keras
import sys
import scipy.stats
import json
import numpy.fft
import time
from decimal import Decimal
import math
import seaborn as sns
%matplotlib inline

In [ ]:
# fix random seed for reproducibility
tf.random.set_seed(1234)

In [ ]:
df = pd.read_excel('/content/sample_data/new_dataset.xlsx')
node1_delay = df[['node1_delay']]

dataset = node1_delay.values

# MA Filter Implementation

In [ ]:
# ASAP
class Metrics(object):
    def __init__(self, values):
        self.set_values( values )

    def set_values(self, values):
        self.values = values
        self.r = self.k = None

    @property
    def kurtosis(self):
        if self.k is None:
            self.k = scipy.stats.kurtosis(self.values)
        return self.k

    @property
    def roughness(self):
        if self.r is None:
            self.r = np.std(np.diff(self.values))
        return self.r

class ACF(Metrics):
    CORR_THRESH = 0.2
    def __init__(self, values, max_lag=None):
        super(ACF, self).__init__(values)
        if max_lag is None:
            max_lag = len(values) / 5
        self.max_lag = int(max_lag)
        self.max_acf = 0.0

        # Calculate autocorrelation via FFT
        # Demean
        demeaned = values - np.mean(values)
        # Pad data to power of 2
        l = int(2.0 ** (int(math.log(len(demeaned),2.0)) + 1))
        padded = np.append(demeaned, ([0.0] * (l - len(demeaned))))
        # FFT and inverse FFT
        F_f = numpy.fft.fft( padded )
        R_t = numpy.fft.ifft( F_f * np.conjugate(F_f) )
        self.correlations = R_t[:int(max_lag)].real / R_t[0].real

        # Find autocorrelation peaks
        self.peaks = []
        if len(self.correlations) >1 :
            positive = self.correlations[1] > self.correlations[0]
            max = 1
            for i in range(2, len(self.correlations)):
                if not positive and self.correlations[i] > self.correlations[i-1]:
                    max = i
                    positive = not positive
                elif positive and self.correlations[i] > self.correlations[max]:
                    max = i
                elif positive and self.correlations[i] < self.correlations[i-1]:
                    if max > 1 and self.correlations[max] > self.CORR_THRESH:
                        self.peaks.append(max)
                        if self.correlations[max] > self.max_acf:
                            self.max_acf = self.correlations[max]
                    positive = not positive
        # If there is no autocorrelation peak within the MAX_WINDOW boundary,
        # try windows from the largest to the smallest
        if len(self.peaks) <= 1:
            self.peaks = range(2, len(self.correlations))

def moving_average(data, _range):
    ret = np.cumsum(data)
    ret[_range:] = ret[int(_range):] - ret[:-int(_range)]
    return ret[int(_range) - 1:] / _range

def SMA(data, _range, slide):
    ret = moving_average(data, int(_range))[::int(slide)]
    return list(ret)

def binary_search(head,tail,data,min_obj,orig_kurt,window_size):
    while head <= tail:
        w = int(round((head + tail) / 2.0))
        smoothed = SMA(data,w,1)
        metrics  = Metrics(smoothed)
        if metrics.kurtosis >= orig_kurt:
            if metrics.roughness < min_obj:
                window_size = w
                min_obj = metrics.roughness
            head = w + 1
        else:
            tail = w - 1
    return window_size

def smooth_ASAP(data, max_window=5, resolution=None):
    data = np.array(data)
    # Preaggregate according to resolution
    slide_size = 1
    window_size = 1
    if resolution and len(data) >= 2 * resolution:
        slide_size = len(data) / resolution
        data = SMA(data, slide_size, slide_size)
    acf         = ACF(data, max_lag=len(data) / max_window)
    peaks       = acf.peaks
    orig_kurt   = acf.kurtosis
    min_obj     = acf.roughness
    lb          = 1
    largest_feasible = -1
    tail = len(data) / max_window
    for i in range(len(peaks) - 1, -1, -1):
        w = peaks[i]

        if w < lb or w == 1:
            break
        elif math.sqrt(1 - acf.correlations[w]) * window_size > math.sqrt(1 - acf.correlations[window_size]) * w:
            continue

        smoothed = SMA(data, w, 1)
        metrics = Metrics(smoothed)
        if metrics.roughness < min_obj and metrics.kurtosis >= orig_kurt:
            min_obj = metrics.roughness
            window_size = w
            lb = round( max(w*math.sqrt( (acf.max_acf -1) / (acf.correlations[w]-1) ), lb) )
    if largest_feasible > 0:
        if largest_feasible < len(peaks) - 2:
            tail = peaks[largest_feasible + 1]
        lb = max(lb, peaks[largest_feasible] + 1)

    window_size = binary_search(lb, tail, data, min_obj, orig_kurt, window_size)
    return window_size, slide_size

# Algorithm to transfer data in tabular-Multivariate format

In [ ]:
# split a multivariate sequence into samples
def create_multivariate_dataset(sequences, look_back):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + look_back
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# ESN Model Library

In [ ]:
def correct_dimensions(s, targetlength):
    """checks the dimensionality of some numeric argument s, broadcasts it
       to the specified length if possible.

    Args:
        s: None, scalar or 1D array
        targetlength: expected length of s

    Returns:
        None if s is None, else numpy vector of length targetlength
    """
    if s is not None:
        s = np.array(s)
        if s.ndim == 0:
            s = np.array([s] * targetlength)
        elif s.ndim == 1:
            if not len(s) == targetlength:
                raise ValueError("arg must have length " + str(targetlength))
        else:
            raise ValueError("Invalid argument")
    return s


def identity(x):
    return x


class ESN():

    def __init__(self, n_inputs, n_outputs, n_reservoir=200,
                 spectral_radius=0.95, sparsity=0, noise=0.001, input_shift=None,
                 input_scaling=None, teacher_forcing=True, feedback_scaling=None,
                 teacher_scaling=None, teacher_shift=None,
                 out_activation=identity, inverse_out_activation=identity,
                 random_state=None, silent=True):
        """
        Args:
            n_inputs: nr of input dimensions
            n_outputs: nr of output dimensions
            n_reservoir: nr of reservoir neurons
            spectral_radius: spectral radius of the recurrent weight matrix
            sparsity: proportion of recurrent weights set to zero
            noise: noise added to each neuron (regularization)
            input_shift: scalar or vector of length n_inputs to add to each
                        input dimension before feeding it to the network.
            input_scaling: scalar or vector of length n_inputs to multiply
                        with each input dimension before feeding it to the netw.
            teacher_forcing: if True, feed the target back into output units
            teacher_scaling: factor applied to the target signal
            teacher_shift: additive term applied to the target signal
            out_activation: output activation function (applied to the readout)
            inverse_out_activation: inverse of the output activation function
            random_state: positive integer seed, np.rand.RandomState object,
                          or None to use numpy's builting RandomState.
            silent: supress messages
        """
        # check for proper dimensionality of all arguments and write them down.
        self.n_inputs = n_inputs
        self.n_reservoir = n_reservoir
        self.n_outputs = n_outputs
        self.spectral_radius = spectral_radius
        self.sparsity = sparsity
        self.noise = noise
        self.input_shift = correct_dimensions(input_shift, n_inputs)
        self.input_scaling = correct_dimensions(input_scaling, n_inputs)

        self.teacher_scaling = teacher_scaling
        self.teacher_shift = teacher_shift

        self.out_activation = out_activation
        self.inverse_out_activation = inverse_out_activation
        self.random_state = random_state

        # the given random_state might be either an actual RandomState object,
        # a seed or None (in which case we use numpy's builtin RandomState)
        if isinstance(random_state, np.random.RandomState):
            self.random_state_ = random_state
        elif random_state:
            try:
                self.random_state_ = np.random.RandomState(random_state)
            except TypeError as e:
                raise Exception("Invalid seed: " + str(e))
        else:
            self.random_state_ = np.random.mtrand._rand

        self.teacher_forcing = teacher_forcing
        self.silent = silent
        self.initweights()

    def initweights(self):
        # initialize recurrent weights:
        # begin with a random matrix centered around zero:
        W = self.random_state_.rand(self.n_reservoir, self.n_reservoir) - 0.5
        # delete the fraction of connections given by (self.sparsity):
        W[self.random_state_.rand(*W.shape) < self.sparsity] = 0
        # compute the spectral radius of these weights:
        radius = np.max(np.abs(np.linalg.eigvals(W)))
        # rescale them to reach the requested spectral radius:
        self.W = W * (self.spectral_radius / radius)

        # random input weights:
        self.W_in = self.random_state_.rand(
            self.n_reservoir, self.n_inputs) * 2 - 1
        # random feedback (teacher forcing) weights:
        self.W_feedb = self.random_state_.rand(
            self.n_reservoir, self.n_outputs) * 2 - 1

    def _update(self, state, input_pattern, output_pattern):
        """performs one update step.

        i.e., computes the next network state by applying the recurrent weights
        to the last state & and feeding in the current input and output patterns
        """
        if self.teacher_forcing:
            preactivation = (np.dot(self.W, state)
                             + np.dot(self.W_in, input_pattern)
                             + np.dot(self.W_feedb, output_pattern))
        else:
            preactivation = (np.dot(self.W, state)
                             + np.dot(self.W_in, input_pattern))
        return (np.tanh(preactivation)
                + self.noise * (self.random_state_.rand(self.n_reservoir) - 0.5))

    def _scale_inputs(self, inputs):
        """for each input dimension j: multiplies by the j'th entry in the
        input_scaling argument, then adds the j'th entry of the input_shift
        argument."""
        if self.input_scaling is not None:
            inputs = np.dot(inputs, np.diag(self.input_scaling))
        if self.input_shift is not None:
            inputs = inputs + self.input_shift
        return inputs

    def _scale_teacher(self, teacher):
        """multiplies the teacher/target signal by the teacher_scaling argument,
        then adds the teacher_shift argument to it."""
        if self.teacher_scaling is not None:
            teacher = teacher * self.teacher_scaling
        if self.teacher_shift is not None:
            teacher = teacher + self.teacher_shift
        return teacher

    def _unscale_teacher(self, teacher_scaled):
        """inverse operation of the _scale_teacher method."""
        if self.teacher_shift is not None:
            teacher_scaled = teacher_scaled - self.teacher_shift
        if self.teacher_scaling is not None:
            teacher_scaled = teacher_scaled / self.teacher_scaling
        return teacher_scaled

    def fit(self, inputs, outputs, inspect=False):
        """
        Collect the network's reaction to training data, train readout weights.

        Args:
            inputs: array of dimensions (N_training_samples x n_inputs)
            outputs: array of dimension (N_training_samples x n_outputs)
            inspect: show a visualisation of the collected reservoir states

        Returns:
            the network's output on the training data, using the trained weights
        """
        # transform any vectors of shape (x,) into vectors of shape (x,1):
        if inputs.ndim < 2:
            inputs = np.reshape(inputs, (len(inputs), -1))
        if outputs.ndim < 2:
            outputs = np.reshape(outputs, (len(outputs), -1))
        # transform input and teacher signal:
        inputs_scaled = self._scale_inputs(inputs)
        teachers_scaled = self._scale_teacher(outputs)

        if not self.silent:
            print("harvesting states...")
        # step the reservoir through the given input,output pairs:
        states = np.zeros((inputs.shape[0], self.n_reservoir))
        for n in range(1, inputs.shape[0]):
            states[n, :] = self._update(states[n - 1], inputs_scaled[n, :],
                                        teachers_scaled[n - 1, :])

        # learn the weights, i.e. find the linear combination of collected
        # network states that is closest to the target output
        if not self.silent:
            print("fitting...")
        # we'll disregard the first few states:
        transient = min(int(inputs.shape[1] / 10), 100)
        # include the raw inputs:
        extended_states = np.hstack((states, inputs_scaled))
        # Solve for W_out:
        self.W_out = np.dot(np.linalg.pinv(extended_states[transient:, :]),
                            self.inverse_out_activation(teachers_scaled[transient:, :])).T

        # remember the last state for later:
        self.laststate = states[-1, :]
        self.lastinput = inputs[-1, :]
        self.lastoutput = teachers_scaled[-1, :]

        # optionally visualize the collected states
        if inspect:
            from matplotlib import pyplot as plt
            # (^-- we depend on matplotlib only if this option is used)
            plt.figure(
                figsize=(states.shape[0] * 0.0025, states.shape[1] * 0.01))
            plt.imshow(extended_states.T, aspect='auto',
                       interpolation='nearest')
            plt.colorbar()

        if not self.silent:
            print("training error:")
        # apply learned weights to the collected states:
        pred_train = self._unscale_teacher(self.out_activation(
            np.dot(extended_states, self.W_out.T)))
        if not self.silent:
            print(np.sqrt(np.mean((pred_train - outputs)**2)))
        return pred_train

    def predict(self, inputs, continuation=True):
        """
        Apply the learned weights to the network's reactions to new input.

        Args:
            inputs: array of dimensions (N_test_samples x n_inputs)
            continuation: if True, start the network from the last training state

        Returns:
            Array of output activations
        """
        if inputs.ndim < 2:
            inputs = np.reshape(inputs, (len(inputs), -1))
        n_samples = inputs.shape[0]

        if continuation:
            laststate = self.laststate
            lastinput = self.lastinput
            lastoutput = self.lastoutput
        else:
            laststate = np.zeros(self.n_reservoir)
            lastinput = np.zeros(self.n_inputs)
            lastoutput = np.zeros(self.n_outputs)

        inputs = np.vstack([lastinput, self._scale_inputs(inputs)])
        states = np.vstack(
            [laststate, np.zeros((n_samples, self.n_reservoir))])
        outputs = np.vstack(
            [lastoutput, np.zeros((n_samples, self.n_outputs))])

        for n in range(n_samples):
            states[
                n + 1, :] = self._update(states[n, :], inputs[n + 1, :], outputs[n, :])
            outputs[n + 1, :] = self.out_activation(np.dot(self.W_out,
                                                           np.concatenate([states[n + 1, :], inputs[n + 1, :]])))

        return self._unscale_teacher(self.out_activation(outputs[1:]))

# Multi-variate per Node

### Finding Best Hyper-parameters

In [ ]:
!pip install -q optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.3 MB/s eta 0:00:00


In [ ]:
import optuna
from optuna.samplers import TPESampler

In [ ]:
node1_delay = df[['node1_delay']]
node2_delay = df[['node2_delay']]
node3_delay = df[['node3_delay']]

node1_delay_dataset = node1_delay.values
node2_delay_dataset = node2_delay.values
node3_delay_dataset = node3_delay.values

window_size1, slide_size1 = smooth_ASAP(node1_delay_dataset, resolution=50)
window_size2, slide_size2 = smooth_ASAP(node2_delay_dataset, resolution=50)
window_size3, slide_size3 = smooth_ASAP(node3_delay_dataset, resolution=50)

print("Window Size1: ", window_size1)
print("Window Size2: ", window_size2)
print("Window Size3: ", window_size3)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size1)
denoised_node2_delay_dataset = moving_average(node2_delay_dataset, window_size2)
denoised_node3_delay_dataset = moving_average(node3_delay_dataset, window_size3)

# Déterminez la longueur minimale des trois séries
min_length = min(len(denoised_node1_delay_dataset), len(denoised_node2_delay_dataset), len(denoised_node3_delay_dataset))

out_seq = np.array([denoised_node1_delay_dataset[i]+denoised_node2_delay_dataset[i]+denoised_node3_delay_dataset[i] for i in range(min_length)])

# convert to [rows, columns] structure
denoised_node1_delay_dataset = denoised_node1_delay_dataset.reshape((len(denoised_node1_delay_dataset), 1))
denoised_node2_delay_dataset = denoised_node2_delay_dataset.reshape((len(denoised_node2_delay_dataset), 1))
denoised_node3_delay_dataset = denoised_node3_delay_dataset.reshape((len(denoised_node3_delay_dataset), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# horizontally stack columns
dataset = np.hstack((denoised_node1_delay_dataset[:min_length], denoised_node2_delay_dataset[:min_length], denoised_node3_delay_dataset[:min_length], out_seq))

# choose a loock_back
look_back = 3
# convert into input/output
X, y = create_multivariate_dataset(dataset, look_back)
print(X.shape, y.shape)

Window Size1:  10
Window Size2:  9
Window Size3:  10
(19988, 3, 4) (19988, 4)


In [ ]:
X[0]

array([[0.26291662, 0.2221311 , 0.19253068, 0.67757839],
       [0.2851563 , 0.22008744, 0.20447593, 0.70971967],
       [0.31522198, 0.22373377, 0.21371023, 0.75266599]])

In [ ]:
y[0]

array([0.31413992, 0.21027635, 0.2119128 , 0.73632908])

In [ ]:
node1_delay = df[['node1_delay']]
node2_delay = df[['node2_delay']]
node3_delay = df[['node3_delay']]

node1_delay_dataset = node1_delay.values
node2_delay_dataset = node2_delay.values
node3_delay_dataset = node3_delay.values

window_size1, slide_size1 = smooth_ASAP(node1_delay_dataset, resolution=50)
window_size2, slide_size2 = smooth_ASAP(node2_delay_dataset, resolution=50)
window_size3, slide_size3 = smooth_ASAP(node3_delay_dataset, resolution=50)

print("Window Size1: ", window_size1)
print("Window Size2: ", window_size2)
print("Window Size3: ", window_size3)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size1)
denoised_node2_delay_dataset = moving_average(node2_delay_dataset, window_size2)
denoised_node3_delay_dataset = moving_average(node3_delay_dataset, window_size3)

# Déterminez la longueur minimale des trois séries
min_length = min(len(denoised_node1_delay_dataset), len(denoised_node2_delay_dataset), len(denoised_node3_delay_dataset))

# convert to [rows, columns] structure
denoised_node1_delay_dataset = denoised_node1_delay_dataset.reshape((len(denoised_node1_delay_dataset), 1))
denoised_node2_delay_dataset = denoised_node2_delay_dataset.reshape((len(denoised_node2_delay_dataset), 1))
denoised_node3_delay_dataset = denoised_node3_delay_dataset.reshape((len(denoised_node3_delay_dataset), 1))

# horizontally stack columns
dataset = np.hstack((denoised_node1_delay_dataset[:min_length], denoised_node2_delay_dataset[:min_length], denoised_node3_delay_dataset[:min_length]))

# Define the objective function for Optuna optimization
def objective(trial):
    # Sample hyperparameters to search

    n_reservoir = trial.suggest_int('n_reservoir', 10, 1000)   # -
    sparsity = trial.suggest_categorical('sparsity', [0.01, 0.1, 0.2, 0.3, 0.4, 0.5])   # -
    spectral_radius = trial.suggest_categorical('spectral_radius', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.25, 10.0])   # - spectral radius of W
    noise = trial.suggest_categorical('noise', [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009])       # - Noise Set
    look_back = trial.suggest_int('look_back', 10, 500)

    # convert into input/output
    X, Y = create_multivariate_dataset(dataset, look_back)

    # Reshape X in 2D Dimention
    X = np.reshape(X, (X.shape[0], -1))

    test_size = int(len(X) * 0.10)

    # Divisez manuellement X et Y en ensembles d'entraînement et de test
    trainX, trainY, testX, testY = X[:-test_size], Y[:-test_size], X[-test_size:], Y[-test_size:]
    print(trainX.shape, trainY.shape)

    model = ESN(n_inputs = trainX.shape[1],
            n_outputs = Y.shape[1],
            n_reservoir = n_reservoir,
            sparsity=sparsity,
            random_state=1234,
            spectral_radius=spectral_radius,
            noise = noise,
            silent = False,
    )

    pred_train = model.fit(trainX, trainY)

    # make predictions
    testPredict = model.predict(testX)
    testPredict = np.array(testPredict)

    node1_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 0], testPredict[:len(testY), 0]))
    node2_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 1], testPredict[:len(testY), 1]))
    node3_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 2], testPredict[:len(testY), 2]))

    print(node1_delay_RMSE, node2_delay_RMSE, node3_delay_RMSE)

    testRMSE = np.sqrt(mean_squared_error(testY, testPredict[:len(testY), :]))

    return node1_delay_RMSE + node2_delay_RMSE + node3_delay_RMSE


# Create the Optuna study
study = optuna.create_study(direction='minimize')

# Record the starting time to generate predictions
start_time = time.time()

# Run the optimization
study.optimize(objective, n_trials=100)

# Record the ending time
end_time = time.time()
elapsed_time = end_time - start_time

print('done')
print("ESN HyperParameters Tuning Elapsed Time : %.5f" % (elapsed_time), "seconds")

# Print the best parameters and corresponding loss
best_params = study.best_params
best_loss = study.best_value
print("Best Parameters: ", best_params)
print("Best Loss: ", best_loss)

[I 2023-09-07 21:33:19,392] A new study created in memory with name: no-name-9b1fcae3-a41a-4aa6-9ebe-3adb346e87a2


Window Size1:  10
Window Size2:  9
Window Size3:  10
(17804, 627) (17804, 3)
harvesting states...
fitting...
training error:
0.010549398487120283


[I 2023-09-07 21:33:36,975] Trial 0 finished with value: 0.031792790089844744 and parameters: {'n_reservoir': 488, 'sparsity': 0.1, 'spectral_radius': 0.6, 'noise': 0.005, 'look_back': 209}. Best is trial 0 with value: 0.031792790089844744.


0.016234862269818583 0.00630381949553961 0.009254108324486554
(17560, 1440) (17560, 3)
harvesting states...
fitting...
training error:
0.010247059692914824


[I 2023-09-07 21:34:03,319] Trial 1 finished with value: 0.031753134659255375 and parameters: {'n_reservoir': 224, 'sparsity': 0.2, 'spectral_radius': 0.8, 'noise': 0.008, 'look_back': 480}. Best is trial 1 with value: 0.031753134659255375.


...

0.01627530960996956 0.006276306215199761 0.009165920169661586
(17722, 900) (17722, 3)
harvesting states...
fitting...
training error:
0.01052027604194091


[I 2023-09-07 21:56:15,501] Trial 98 finished with value: 0.031365726389302184 and parameters: {'n_reservoir': 140, 'sparsity': 0.3, 'spectral_radius': 0.7, 'noise': 0.006, 'look_back': 300}. Best is trial 47 with value: 0.031143968236715815.


0.016091525444839303 0.006208409471612159 0.009065791472850724
(17697, 984) (17697, 3)
harvesting states...
fitting...


[I 2023-09-07 21:56:26,221] Trial 99 finished with value: 0.031209073007350326 and parameters: {'n_reservoir': 80, 'sparsity': 0.2, 'spectral_radius': 10.0, 'noise': 0.004, 'look_back': 328}. Best is trial 47 with value: 0.031143968236715815.


training error:
0.01049380153597709
0.015981446524220337 0.0061812168493684955 0.009046409633761491
done
ESN HyperParameters Tuning Elapsed Time : 1386.82596 seconds
Best Parameters:  {'n_reservoir': 11, 'sparsity': 0.2, 'spectral_radius': 10.0, 'noise': 0.006, 'look_back': 289}
Best Loss:  0.031143968236715815


## Prédiction multivariée des caractéristiques d’un nœud unique

### Node1

In [ ]:
node1_delay = df[['node1_delay']]
node2_delay = df[['node1_cpu']]
node3_delay = df[['node1_ram']]

node1_delay_dataset = node1_delay.values
node2_delay_dataset = node2_delay.values
node3_delay_dataset = node3_delay.values

window_size1, slide_size1 = smooth_ASAP(node1_delay_dataset, resolution=50)
window_size2, slide_size2 = smooth_ASAP(node2_delay_dataset, resolution=50)
window_size3, slide_size3 = smooth_ASAP(node3_delay_dataset, resolution=50)

print("Window Size1: ", window_size1)
print("Window Size2: ", window_size2)
print("Window Size3: ", window_size3)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size1)
denoised_node2_delay_dataset = moving_average(node2_delay_dataset, window_size2)
denoised_node3_delay_dataset = moving_average(node3_delay_dataset, window_size3)

# Déterminez la longueur minimale des trois séries
min_length = min(len(denoised_node1_delay_dataset), len(denoised_node2_delay_dataset), len(denoised_node3_delay_dataset))

# convert to [rows, columns] structure
denoised_node1_delay_dataset = denoised_node1_delay_dataset.reshape((len(denoised_node1_delay_dataset), 1))
denoised_node2_delay_dataset = denoised_node2_delay_dataset.reshape((len(denoised_node2_delay_dataset), 1))
denoised_node3_delay_dataset = denoised_node3_delay_dataset.reshape((len(denoised_node3_delay_dataset), 1))

# horizontally stack columns
dataset = np.hstack((denoised_node1_delay_dataset[:min_length], denoised_node2_delay_dataset[:min_length], denoised_node3_delay_dataset[:min_length]))

# Define the objective function for Optuna optimization
def objective(trial):
    # Sample hyperparameters to search

    n_reservoir = trial.suggest_int('n_reservoir', 10, 1000)   # -
    sparsity = trial.suggest_categorical('sparsity', [0.01, 0.1, 0.2, 0.3, 0.4, 0.5])   # -
    spectral_radius = trial.suggest_categorical('spectral_radius', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.25, 10.0])   # - spectral radius of W
    noise = trial.suggest_categorical('noise', [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009])       # - Noise Set
    look_back = trial.suggest_int('look_back', 10, 500)

    # convert into input/output
    X, Y = create_multivariate_dataset(dataset, look_back)

    # Reshape X in 2D Dimention
    X = np.reshape(X, (X.shape[0], -1))

    test_size = int(len(X) * 0.10)

    # Divisez manuellement X et Y en ensembles d'entraînement et de test
    trainX, trainY, testX, testY = X[:-test_size], Y[:-test_size], X[-test_size:], Y[-test_size:]
    print(trainX.shape, trainY.shape)

    model = ESN(n_inputs = trainX.shape[1],
            n_outputs = Y.shape[1],
            n_reservoir = n_reservoir,
            sparsity=sparsity,
            random_state=1234,
            spectral_radius=spectral_radius,
            noise = noise,
            silent = False,
    )

    pred_train = model.fit(trainX, trainY)

    # make predictions
    testPredict = model.predict(testX)
    testPredict = np.array(testPredict)

    node1_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 0], testPredict[:len(testY), 0]))
    node2_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 1], testPredict[:len(testY), 1]))
    node3_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 2], testPredict[:len(testY), 2]))

    print(node1_delay_RMSE, node2_delay_RMSE, node3_delay_RMSE)

    testRMSE = np.sqrt(mean_squared_error(testY, testPredict[:len(testY), :]))

    return testRMSE


# Create the Optuna study
study = optuna.create_study(direction='minimize')

# Record the starting time to generate predictions
start_time = time.time()

# Run the optimization
study.optimize(objective, n_trials=100)

# Record the ending time
end_time = time.time()
elapsed_time = end_time - start_time

print('done')
print("ESN HyperParameters Tuning Elapsed Time : %.5f" % (elapsed_time), "seconds")

# Print the best parameters and corresponding loss
best_params = study.best_params
best_loss = study.best_value
print("Best Parameters: ", best_params)
print("Best Loss: ", best_loss)

[I 2023-09-07 15:58:08,991] A new study created in memory with name: no-name-706c58bc-702c-4147-8689-7dfda4750204


Window Size1:  10
Window Size2:  3
Window Size3:  10
(17964, 93) (17964, 3)
harvesting states...
fitting...
training error:
0.05011678409272601


[I 2023-09-07 15:58:38,516] Trial 0 finished with value: 0.05414653153619244 and parameters: {'n_reservoir': 962, 'sparsity': 0.5, 'spectral_radius': 1.25, 'noise': 0.004, 'look_back': 31}. Best is trial 0 with value: 0.05414653153619244.


0.017588000344018324 0.09044090702987906 0.017511288122557485
(17878, 381) (17878, 3)
harvesting states...
fitting...


[I 2023-09-07 15:58:41,319] Trial 1 finished with value: 0.05167232750360344 and parameters: {'n_reservoir': 57, 'sparsity': 0.4, 'spectral_radius': 0.9, 'noise': 0.003, 'look_back': 127}. Best is trial 1 with value: 0.05167232750360344.


...

0.01838391656323454 0.09026384626764775 0.01752598288664785
(17865, 423) (17865, 3)
harvesting states...
fitting...


[I 2023-09-07 16:10:40,396] Trial 98 finished with value: 0.05179256620098381 and parameters: {'n_reservoir': 113, 'sparsity': 0.4, 'spectral_radius': 0.7, 'noise': 0.006, 'look_back': 141}. Best is trial 91 with value: 0.051449476757285936.


training error:
0.04965629588532034
0.01608740243801222 0.08687124525371408 0.01555609115158325
(17930, 207) (17930, 3)
harvesting states...
fitting...
training error:
0.05033812289749446


[I 2023-09-07 16:10:43,453] Trial 99 finished with value: 0.05160478768540345 and parameters: {'n_reservoir': 163, 'sparsity': 0.1, 'spectral_radius': 0.1, 'noise': 0.001, 'look_back': 69}. Best is trial 91 with value: 0.051449476757285936.


0.016504690013392793 0.08636302532063514 0.016068148654896367
done
ESN HyperParameters Tuning Elapsed Time : 754.46129 seconds
Best Parameters:  {'n_reservoir': 60, 'sparsity': 0.4, 'spectral_radius': 0.8, 'noise': 0.001, 'look_back': 91}
Best Loss:  0.051449476757285936


In [ ]:
n_reservoir = 60
sparsity = 0.4
spectral_radius = 0.8
noise = 0.001
look_back = 91
# Best Loss:  0.051449476757285936

In [ ]:
node1_delay = df[['node1_delay']]
node2_delay = df[['node1_cpu']]
node3_delay = df[['node1_ram']]

node1_delay_dataset = node1_delay.values
node2_delay_dataset = node2_delay.values
node3_delay_dataset = node3_delay.values

window_size1, slide_size1 = smooth_ASAP(node1_delay_dataset, resolution=50)
window_size2, slide_size2 = smooth_ASAP(node2_delay_dataset, resolution=50)
window_size3, slide_size3 = smooth_ASAP(node3_delay_dataset, resolution=50)

print("Window Size1: ", window_size1)
print("Window Size2: ", window_size2)
print("Window Size3: ", window_size3)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size1)
denoised_node2_delay_dataset = moving_average(node2_delay_dataset, window_size2)
denoised_node3_delay_dataset = moving_average(node3_delay_dataset, window_size3)

# Déterminez la longueur minimale des trois séries
min_length = min(len(denoised_node1_delay_dataset), len(denoised_node2_delay_dataset), len(denoised_node3_delay_dataset))

# convert to [rows, columns] structure
denoised_node1_delay_dataset = denoised_node1_delay_dataset.reshape((len(denoised_node1_delay_dataset), 1))
denoised_node2_delay_dataset = denoised_node2_delay_dataset.reshape((len(denoised_node2_delay_dataset), 1))
denoised_node3_delay_dataset = denoised_node3_delay_dataset.reshape((len(denoised_node3_delay_dataset), 1))

# horizontally stack columns
dataset = np.hstack((denoised_node1_delay_dataset[:min_length], denoised_node2_delay_dataset[:min_length], denoised_node3_delay_dataset[:min_length]))

# convert into input/output
X, Y = create_multivariate_dataset(dataset, look_back)

# Reshape X in 2D Dimention
X = np.reshape(X, (X.shape[0], -1))

test_size = int(len(X) * 0.10)

# Divisez manuellement X et Y en ensembles d'entraînement et de test
trainX, trainY, testX, testY = X[:-test_size], Y[:-test_size], X[-test_size:], Y[-test_size:]
print(trainX.shape, trainY.shape)

model = ESN(n_inputs = trainX.shape[1],
        n_outputs = Y.shape[1],
        n_reservoir = n_reservoir,
        sparsity=sparsity,
        random_state=1234,
        spectral_radius=spectral_radius,
        noise = noise,
        silent = False,
)

# Record the starting time to training the model
training_start_time = time.time()

pred_train = model.fit(trainX, trainY)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time

# Record the starting time to generate predictions
predictions_start_time = time.time()

# make predictions
testPredict = model.predict(testX)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time

testPredict = np.array(testPredict)

node1_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 0], testPredict[:len(testY), 0]))
node2_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 1], testPredict[:len(testY), 1]))
node3_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 2], testPredict[:len(testY), 2]))

node1_delay_MAE = mean_absolute_error(testY[:, 0], testPredict[:len(testY), 0])
node2_delay_MAE = mean_absolute_error(testY[:, 1], testPredict[:len(testY), 1])
node3_delay_MAE = mean_absolute_error(testY[:, 2], testPredict[:len(testY), 2])



testRMSE = np.sqrt(mean_squared_error(testY, testPredict[:len(testY), :]))
testMAE = mean_absolute_error(testY, testPredict[:len(testY), :])

print("Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

print('Node1 Delay Test RMSE : %.5f' % (node1_delay_RMSE))
print('Node1 CPU Test RMSE : %.5f' % (node2_delay_RMSE))
print('Node1 RAM Test RMSE : %.5f' % (node3_delay_RMSE))
print('Global Test RMSE : %.5f' % (testRMSE))

print('Node1 Delay Test MAE : %.5f' % (node1_delay_MAE))
print('Node1 CPU Test MAE : %.5f' % (node2_delay_MAE))
print('Node1 RAM Test MAE : %.5f' % (node3_delay_MAE))
print('Global Test MAE : %.5f' % (testMAE))

print("Model Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size1:  10
Window Size2:  3
Window Size3:  10
(17910, 273) (17910, 3)
harvesting states...
fitting...
training error:
0.05022954839863735
Model Training Elapsed Time : 2.44192 seconds
Node1 Delay Test RMSE : 0.01624
Node1 CPU Test RMSE : 0.08619
Node1 RAM Test RMSE : 0.01578
Global Test RMSE : 0.05145
Node1 Delay Test MAE : 0.01391
Node1 CPU Test MAE : 0.07506
Node1 RAM Test MAE : 0.01338
Global Test MAE : 0.03412
Model Elapsed Time To generate Predictions : 0.10297 seconds


### Node2

In [ ]:
node1_delay = df[['node2_delay']]
node2_delay = df[['node2_cpu']]
node3_delay = df[['node2_ram']]

node1_delay_dataset = node1_delay.values
node2_delay_dataset = node2_delay.values
node3_delay_dataset = node3_delay.values

window_size1, slide_size1 = smooth_ASAP(node1_delay_dataset, resolution=50)
window_size2, slide_size2 = smooth_ASAP(node2_delay_dataset, resolution=50)
window_size3, slide_size3 = smooth_ASAP(node3_delay_dataset, resolution=50)

print("Window Size1: ", window_size1)
print("Window Size2: ", window_size2)
print("Window Size3: ", window_size3)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size1)
denoised_node2_delay_dataset = moving_average(node2_delay_dataset, window_size2)
denoised_node3_delay_dataset = moving_average(node3_delay_dataset, window_size3)

# Déterminez la longueur minimale des trois séries
min_length = min(len(denoised_node1_delay_dataset), len(denoised_node2_delay_dataset), len(denoised_node3_delay_dataset))

# convert to [rows, columns] structure
denoised_node1_delay_dataset = denoised_node1_delay_dataset.reshape((len(denoised_node1_delay_dataset), 1))
denoised_node2_delay_dataset = denoised_node2_delay_dataset.reshape((len(denoised_node2_delay_dataset), 1))
denoised_node3_delay_dataset = denoised_node3_delay_dataset.reshape((len(denoised_node3_delay_dataset), 1))

# horizontally stack columns
dataset = np.hstack((denoised_node1_delay_dataset[:min_length], denoised_node2_delay_dataset[:min_length], denoised_node3_delay_dataset[:min_length]))

# Define the objective function for Optuna optimization
def objective(trial):
    # Sample hyperparameters to search

    n_reservoir = trial.suggest_int('n_reservoir', 10, 1000)   # -
    sparsity = trial.suggest_categorical('sparsity', [0.01, 0.1, 0.2, 0.3, 0.4, 0.5])   # -
    spectral_radius = trial.suggest_categorical('spectral_radius', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.25, 10.0])   # - spectral radius of W
    noise = trial.suggest_categorical('noise', [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009])       # - Noise Set
    look_back = trial.suggest_int('look_back', 10, 500)

    # convert into input/output
    X, Y = create_multivariate_dataset(dataset, look_back)

    # Reshape X in 2D Dimention
    X = np.reshape(X, (X.shape[0], -1))

    test_size = int(len(X) * 0.10)

    # Divisez manuellement X et Y en ensembles d'entraînement et de test
    trainX, trainY, testX, testY = X[:-test_size], Y[:-test_size], X[-test_size:], Y[-test_size:]
    print(trainX.shape, trainY.shape)

    model = ESN(n_inputs = trainX.shape[1],
            n_outputs = Y.shape[1],
            n_reservoir = n_reservoir,
            sparsity=sparsity,
            random_state=1234,
            spectral_radius=spectral_radius,
            noise = noise,
            silent = False,
    )

    pred_train = model.fit(trainX, trainY)

    # make predictions
    testPredict = model.predict(testX)
    testPredict = np.array(testPredict)

    node1_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 0], testPredict[:len(testY), 0]))
    node2_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 1], testPredict[:len(testY), 1]))
    node3_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 2], testPredict[:len(testY), 2]))

    print(node1_delay_RMSE, node2_delay_RMSE, node3_delay_RMSE)

    testRMSE = np.sqrt(mean_squared_error(testY, testPredict[:len(testY), :]))

    return testRMSE


# Create the Optuna study
study = optuna.create_study(direction='minimize')

# Record the starting time to generate predictions
start_time = time.time()

# Run the optimization
study.optimize(objective, n_trials=100)

# Record the ending time
end_time = time.time()
elapsed_time = end_time - start_time

print('done')
print("ESN HyperParameters Tuning Elapsed Time : %.5f" % (elapsed_time), "seconds")

# Print the best parameters and corresponding loss
best_params = study.best_params
best_loss = study.best_value
print("Best Parameters: ", best_params)
print("Best Loss: ", best_loss)

[I 2023-09-08 09:41:53,260] A new study created in memory with name: no-name-c8f5a272-357c-4189-a544-683449835b89


Window Size1:  9
Window Size2:  1
Window Size3:  6
(17932, 204) (17932, 3)
harvesting states...
fitting...
training error:
0.14093897322921564


[I 2023-09-08 09:42:59,319] Trial 0 finished with value: 0.1493993122404889 and parameters: {'n_reservoir': 979, 'sparsity': 0.4, 'spectral_radius': 1, 'noise': 0.003, 'look_back': 68}. Best is trial 0 with value: 0.1493993122404889.


0.006505237376267414 0.2572377206182553 0.02732947989984561
(17610, 1278) (17610, 3)
harvesting states...
fitting...
training error:
0.139696596387994


[I 2023-09-08 09:43:34,665] Trial 1 finished with value: 0.15104790088902684 and parameters: {'n_reservoir': 144, 'sparsity': 0.01, 'spectral_radius': 0.3, 'noise': 0.003, 'look_back': 426}. Best is trial 0 with value: 0.1493993122404889.


...

0.006193499817042425 0.2503668666572078 0.02651915045893311
(17938, 183) (17938, 3)
harvesting states...
fitting...


[I 2023-09-08 09:54:30,658] Trial 98 finished with value: 0.14495822197692135 and parameters: {'n_reservoir': 27, 'sparsity': 0.3, 'spectral_radius': 0.2, 'noise': 0.003, 'look_back': 61}. Best is trial 41 with value: 0.1447834923427937.


training error:
0.14519134811571333
0.006306977782527 0.2495537041324932 0.02686687814268706
(17927, 222) (17927, 3)
harvesting states...
fitting...


[I 2023-09-08 09:54:32,544] Trial 99 finished with value: 0.14540820705188007 and parameters: {'n_reservoir': 24, 'sparsity': 0.2, 'spectral_radius': 0.2, 'noise': 0.006, 'look_back': 74}. Best is trial 41 with value: 0.1447834923427937.


training error:
0.1449156481159834
0.006276210276201122 0.2503564764918596 0.026699885716552046
done
ESN HyperParameters Tuning Elapsed Time : 759.27651 seconds
Best Parameters:  {'n_reservoir': 43, 'sparsity': 0.2, 'spectral_radius': 0.3, 'noise': 0.005, 'look_back': 54}
Best Loss:  0.1447834923427937


In [ ]:
# Best Params
n_reservoir = 43
sparsity = 0.2
spectral_radius = 0.3
noise = 0.005
look_back = 54

In [ ]:
node1_delay = df[['node2_delay']]
node2_delay = df[['node2_cpu']]
node3_delay = df[['node2_ram']]

node1_delay_dataset = node1_delay.values
node2_delay_dataset = node2_delay.values
node3_delay_dataset = node3_delay.values

window_size1, slide_size1 = smooth_ASAP(node1_delay_dataset, resolution=50)
window_size2, slide_size2 = smooth_ASAP(node2_delay_dataset, resolution=100)
window_size3, slide_size3 = smooth_ASAP(node3_delay_dataset, resolution=50)

print("Window Size1: ", window_size1)
print("Window Size2: ", window_size2)
print("Window Size3: ", window_size3)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size1)
denoised_node2_delay_dataset = moving_average(node2_delay_dataset, window_size2)
denoised_node3_delay_dataset = moving_average(node3_delay_dataset, window_size3)

# Déterminez la longueur minimale des trois séries
min_length = min(len(denoised_node1_delay_dataset), len(denoised_node2_delay_dataset), len(denoised_node3_delay_dataset))

# convert to [rows, columns] structure
denoised_node1_delay_dataset = denoised_node1_delay_dataset.reshape((len(denoised_node1_delay_dataset), 1))
denoised_node2_delay_dataset = denoised_node2_delay_dataset.reshape((len(denoised_node2_delay_dataset), 1))
denoised_node3_delay_dataset = denoised_node3_delay_dataset.reshape((len(denoised_node3_delay_dataset), 1))

# horizontally stack columns
dataset = np.hstack((denoised_node1_delay_dataset[:min_length], denoised_node2_delay_dataset[:min_length], denoised_node3_delay_dataset[:min_length]))

# convert into input/output
X, Y = create_multivariate_dataset(dataset, look_back)

# Reshape X in 2D Dimention
X = np.reshape(X, (X.shape[0], -1))

test_size = int(len(X) * 0.10)

# Divisez manuellement X et Y en ensembles d'entraînement et de test
trainX, trainY, testX, testY = X[:-test_size], Y[:-test_size], X[-test_size:], Y[-test_size:]
print(trainX.shape, trainY.shape)

model = ESN(n_inputs = trainX.shape[1],
        n_outputs = Y.shape[1],
        n_reservoir = n_reservoir,
        sparsity=sparsity,
        random_state=1234,
        spectral_radius=spectral_radius,
        noise = noise,
        silent = False,
)

# Record the starting time to training the model
training_start_time = time.time()

pred_train = model.fit(trainX, trainY)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time

# Record the starting time to generate predictions
predictions_start_time = time.time()

# make predictions
testPredict = model.predict(testX)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time

testPredict = np.array(testPredict)

node1_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 0], testPredict[:len(testY), 0]))
node2_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 1], testPredict[:len(testY), 1]))
node3_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 2], testPredict[:len(testY), 2]))

node1_delay_MAE = mean_absolute_error(testY[:, 0], testPredict[:len(testY), 0])
node2_delay_MAE = mean_absolute_error(testY[:, 1], testPredict[:len(testY), 1])
node3_delay_MAE = mean_absolute_error(testY[:, 2], testPredict[:len(testY), 2])



testRMSE = np.sqrt(mean_squared_error(testY, testPredict[:len(testY), :]))
testMAE = mean_absolute_error(testY, testPredict[:len(testY), :])

print("Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

print('Node2 Delay Test RMSE : %.5f' % (node1_delay_RMSE))
print('Node2 CPU Test RMSE : %.5f' % (node2_delay_RMSE))
print('Node2 RAM Test RMSE : %.5f' % (node3_delay_RMSE))
print('Global Test RMSE : %.5f' % (testRMSE))

print('Node2 Delay Test MAE : %.5f' % (node1_delay_MAE))
print('Node2 CPU Test MAE : %.5f' % (node2_delay_MAE))
print('Node2 RAM Test MAE : %.5f' % (node3_delay_MAE))
print('Global Test MAE : %.5f' % (testMAE))

print("Model Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size1:  9
Window Size2:  2
Window Size3:  6
(17739, 849) (17739, 3)
harvesting states...
fitting...
training error:
0.07237123477865909
Model Training Elapsed Time : 8.07898 seconds
Node2 Delay Test RMSE : 0.00620
Node2 CPU Test RMSE : 0.12815
Node2 RAM Test RMSE : 0.02637
Global Test RMSE : 0.07562
Node2 Delay Test MAE : 0.00531
Node2 CPU Test MAE : 0.10996
Node2 RAM Test MAE : 0.02246
Global Test MAE : 0.04591
Model Elapsed Time To generate Predictions : 0.08289 seconds


### Node3

In [ ]:
node1_delay = df[['node3_delay']]
node2_delay = df[['node3_cpu']]
node3_delay = df[['node3_ram']]

node1_delay_dataset = node1_delay.values
node2_delay_dataset = node2_delay.values
node3_delay_dataset = node3_delay.values

window_size1, slide_size1 = smooth_ASAP(node1_delay_dataset, resolution=50)
window_size2, slide_size2 = smooth_ASAP(node2_delay_dataset, resolution=50)
window_size3, slide_size3 = smooth_ASAP(node3_delay_dataset, resolution=50)

print("Window Size1: ", window_size1)
print("Window Size2: ", window_size2)
print("Window Size3: ", window_size3)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size1)
denoised_node2_delay_dataset = moving_average(node2_delay_dataset, window_size2)
denoised_node3_delay_dataset = moving_average(node3_delay_dataset, window_size3)

# Déterminez la longueur minimale des trois séries
min_length = min(len(denoised_node1_delay_dataset), len(denoised_node2_delay_dataset), len(denoised_node3_delay_dataset))

# convert to [rows, columns] structure
denoised_node1_delay_dataset = denoised_node1_delay_dataset.reshape((len(denoised_node1_delay_dataset), 1))
denoised_node2_delay_dataset = denoised_node2_delay_dataset.reshape((len(denoised_node2_delay_dataset), 1))
denoised_node3_delay_dataset = denoised_node3_delay_dataset.reshape((len(denoised_node3_delay_dataset), 1))

# horizontally stack columns
dataset = np.hstack((denoised_node1_delay_dataset[:min_length], denoised_node2_delay_dataset[:min_length], denoised_node3_delay_dataset[:min_length]))

# Define the objective function for Optuna optimization
def objective(trial):
    # Sample hyperparameters to search

    n_reservoir = trial.suggest_int('n_reservoir', 10, 1000)   # -
    sparsity = trial.suggest_categorical('sparsity', [0.01, 0.1, 0.2, 0.3, 0.4, 0.5])   # -
    spectral_radius = trial.suggest_categorical('spectral_radius', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.25, 10.0])   # - spectral radius of W
    noise = trial.suggest_categorical('noise', [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009])       # - Noise Set
    look_back = trial.suggest_int('look_back', 10, 500)

    # convert into input/output
    X, Y = create_multivariate_dataset(dataset, look_back)

    # Reshape X in 2D Dimention
    X = np.reshape(X, (X.shape[0], -1))

    test_size = int(len(X) * 0.10)

    # Divisez manuellement X et Y en ensembles d'entraînement et de test
    trainX, trainY, testX, testY = X[:-test_size], Y[:-test_size], X[-test_size:], Y[-test_size:]
    print(trainX.shape, trainY.shape)

    model = ESN(n_inputs = trainX.shape[1],
            n_outputs = Y.shape[1],
            n_reservoir = n_reservoir,
            sparsity=sparsity,
            random_state=1234,
            spectral_radius=spectral_radius,
            noise = noise,
            silent = False,
    )

    pred_train = model.fit(trainX, trainY)

    # make predictions
    testPredict = model.predict(testX)
    testPredict = np.array(testPredict)

    node1_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 0], testPredict[:len(testY), 0]))
    node2_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 1], testPredict[:len(testY), 1]))
    node3_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 2], testPredict[:len(testY), 2]))

    print(node1_delay_RMSE, node2_delay_RMSE, node3_delay_RMSE)

    testRMSE = np.sqrt(mean_squared_error(testY, testPredict[:len(testY), :]))

    return testRMSE


# Create the Optuna study
study = optuna.create_study(direction='minimize')

# Record the starting time to generate predictions
start_time = time.time()

# Run the optimization
study.optimize(objective, n_trials=100)

# Record the ending time
end_time = time.time()
elapsed_time = end_time - start_time

print('done')
print("ESN HyperParameters Tuning Elapsed Time : %.5f" % (elapsed_time), "seconds")

# Print the best parameters and corresponding loss
best_params = study.best_params
best_loss = study.best_value
print("Best Parameters: ", best_params)
print("Best Loss: ", best_loss)

[I 2023-09-08 09:54:34,081] A new study created in memory with name: no-name-c184f3b8-0146-453f-b45b-22d1559dc093


Window Size1:  10
Window Size2:  10
Window Size3:  9
(17870, 408) (17870, 3)
harvesting states...
fitting...
training error:
0.01809602392763161


[I 2023-09-08 09:54:51,667] Trial 0 finished with value: 0.019215455548988868 and parameters: {'n_reservoir': 634, 'sparsity': 0.3, 'spectral_radius': 1.1, 'noise': 0.008, 'look_back': 136}. Best is trial 0 with value: 0.019215455548988868.


0.009242847829932845 0.026631199396927256 0.01769322408548843
(17829, 543) (17829, 3)
harvesting states...
fitting...
training error:
0.017828606604792466


[I 2023-09-08 09:55:41,549] Trial 1 finished with value: 0.019121312333783936 and parameters: {'n_reservoir': 762, 'sparsity': 0.1, 'spectral_radius': 1.1, 'noise': 0.009, 'look_back': 181}. Best is trial 1 with value: 0.019121312333783936.


...

0.009029732922732507 0.026109333786566675 0.017158478392717405
(17876, 387) (17876, 3)
harvesting states...
fitting...


[I 2023-09-08 10:16:34,889] Trial 98 finished with value: 0.018860945895025506 and parameters: {'n_reservoir': 33, 'sparsity': 0.2, 'spectral_radius': 0.3, 'noise': 0.006, 'look_back': 129}. Best is trial 11 with value: 0.01869371959773456.


training error:
0.0184595181904873
0.009046986027915994 0.026206420849324508 0.0172795078125513
(17720, 909) (17720, 3)
harvesting states...
fitting...
training error:
0.017757704410889558


[I 2023-09-08 10:16:47,756] Trial 99 finished with value: 0.01886485760729181 and parameters: {'n_reservoir': 180, 'sparsity': 0.1, 'spectral_radius': 0.8, 'noise': 0.002, 'look_back': 303}. Best is trial 11 with value: 0.01869371959773456.


0.009096279613941846 0.02620351423713271 0.017270845272874064
done
ESN HyperParameters Tuning Elapsed Time : 1333.68026 seconds
Best Parameters:  {'n_reservoir': 10, 'sparsity': 0.2, 'spectral_radius': 1, 'noise': 0.005, 'look_back': 283}
Best Loss:  0.01869371959773456


In [ ]:
# Best Params
n_reservoir = 10
sparsity = 0.2
spectral_radius = 1
noise = 0.005
look_back = 283

In [ ]:
node1_delay = df[['node3_delay']]
node2_delay = df[['node3_cpu']]
node3_delay = df[['node3_ram']]

node1_delay_dataset = node1_delay.values
node2_delay_dataset = node2_delay.values
node3_delay_dataset = node3_delay.values

window_size1, slide_size1 = smooth_ASAP(node1_delay_dataset, resolution=50)
window_size2, slide_size2 = smooth_ASAP(node2_delay_dataset, resolution=50)
window_size3, slide_size3 = smooth_ASAP(node3_delay_dataset, resolution=50)

print("Window Size1: ", window_size1)
print("Window Size2: ", window_size2)
print("Window Size3: ", window_size3)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size1)
denoised_node2_delay_dataset = moving_average(node2_delay_dataset, window_size2)
denoised_node3_delay_dataset = moving_average(node3_delay_dataset, window_size3)

# Déterminez la longueur minimale des trois séries
min_length = min(len(denoised_node1_delay_dataset), len(denoised_node2_delay_dataset), len(denoised_node3_delay_dataset))

# convert to [rows, columns] structure
denoised_node1_delay_dataset = denoised_node1_delay_dataset.reshape((len(denoised_node1_delay_dataset), 1))
denoised_node2_delay_dataset = denoised_node2_delay_dataset.reshape((len(denoised_node2_delay_dataset), 1))
denoised_node3_delay_dataset = denoised_node3_delay_dataset.reshape((len(denoised_node3_delay_dataset), 1))

# horizontally stack columns
dataset = np.hstack((denoised_node1_delay_dataset[:min_length], denoised_node2_delay_dataset[:min_length], denoised_node3_delay_dataset[:min_length]))

# convert into input/output
X, Y = create_multivariate_dataset(dataset, look_back)

# Reshape X in 2D Dimention
X = np.reshape(X, (X.shape[0], -1))

test_size = int(len(X) * 0.10)

# Divisez manuellement X et Y en ensembles d'entraînement et de test
trainX, trainY, testX, testY = X[:-test_size], Y[:-test_size], X[-test_size:], Y[-test_size:]
print(trainX.shape, trainY.shape)

model = ESN(n_inputs = trainX.shape[1],
        n_outputs = Y.shape[1],
        n_reservoir = n_reservoir,
        sparsity=sparsity,
        random_state=1234,
        spectral_radius=spectral_radius,
        noise = noise,
        silent = False,
)

# Record the starting time to training the model
training_start_time = time.time()

pred_train = model.fit(trainX, trainY)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time

# Record the starting time to generate predictions
predictions_start_time = time.time()

# make predictions
testPredict = model.predict(testX)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time

testPredict = np.array(testPredict)

node1_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 0], testPredict[:len(testY), 0]))
node2_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 1], testPredict[:len(testY), 1]))
node3_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 2], testPredict[:len(testY), 2]))

node1_delay_MAE = mean_absolute_error(testY[:, 0], testPredict[:len(testY), 0])
node2_delay_MAE = mean_absolute_error(testY[:, 1], testPredict[:len(testY), 1])
node3_delay_MAE = mean_absolute_error(testY[:, 2], testPredict[:len(testY), 2])



testRMSE = np.sqrt(mean_squared_error(testY, testPredict[:len(testY), :]))
testMAE = mean_absolute_error(testY, testPredict[:len(testY), :])

print("Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

print('Node3 Delay Test RMSE : %.5f' % (node1_delay_RMSE))
print('Node3 CPU Test RMSE : %.5f' % (node2_delay_RMSE))
print('Node3 RAM Test RMSE : %.5f' % (node3_delay_RMSE))
print('Global Test RMSE : %.5f' % (testRMSE))

print('Node3 Delay Test MAE : %.5f' % (node1_delay_MAE))
print('Node3 CPU Test MAE : %.5f' % (node2_delay_MAE))
print('Node3 RAM Test MAE : %.5f' % (node3_delay_MAE))
print('Global Test MAE : %.5f' % (testMAE))

print("Model Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size1:  10
Window Size2:  10
Window Size3:  9
(17738, 849) (17738, 3)
harvesting states...
fitting...
training error:
0.017910918661375026
Model Training Elapsed Time : 7.68983 seconds
Node3 Delay Test RMSE : 0.00901
Node3 CPU Test RMSE : 0.02597
Node3 RAM Test RMSE : 0.01711
Global Test RMSE : 0.01869
Node3 Delay Test MAE : 0.00769
Node3 CPU Test MAE : 0.02217
Node3 RAM Test MAE : 0.01443
Global Test MAE : 0.01476
Model Elapsed Time To generate Predictions : 0.08878 seconds


In [ ]:
node1_delay = df[['node1_delay']]
node2_delay = df[['node1_x']]
node3_delay = df[['node1_y']]

node1_delay_dataset = node1_delay.values
node2_delay_dataset = node2_delay.values
node3_delay_dataset = node3_delay.values

window_size1, slide_size1 = smooth_ASAP(node1_delay_dataset, resolution=50)
window_size2, slide_size2 = smooth_ASAP(node2_delay_dataset, resolution=50)
window_size3, slide_size3 = smooth_ASAP(node3_delay_dataset, resolution=50)

print("Window Size1: ", window_size1)
print("Window Size2: ", window_size2)
print("Window Size3: ", window_size3)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size1)
denoised_node2_delay_dataset = moving_average(node2_delay_dataset, window_size2)
denoised_node3_delay_dataset = moving_average(node3_delay_dataset, window_size3)

# Déterminez la longueur minimale des trois séries
min_length = min(len(denoised_node1_delay_dataset), len(denoised_node2_delay_dataset), len(denoised_node3_delay_dataset))

# convert to [rows, columns] structure
denoised_node1_delay_dataset = denoised_node1_delay_dataset.reshape((len(denoised_node1_delay_dataset), 1))
denoised_node2_delay_dataset = denoised_node2_delay_dataset.reshape((len(denoised_node2_delay_dataset), 1))
denoised_node3_delay_dataset = denoised_node3_delay_dataset.reshape((len(denoised_node3_delay_dataset), 1))

# horizontally stack columns
dataset = np.hstack((denoised_node1_delay_dataset[:min_length], denoised_node2_delay_dataset[:min_length], denoised_node3_delay_dataset[:min_length]))

# Define the objective function for Optuna optimization
def objective(trial):
    # Sample hyperparameters to search

    n_reservoir = trial.suggest_int('n_reservoir', 10, 1000)   # -
    sparsity = trial.suggest_categorical('sparsity', [0.01, 0.1, 0.2, 0.3, 0.4, 0.5])   # -
    spectral_radius = trial.suggest_categorical('spectral_radius', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.25, 10.0])   # - spectral radius of W
    noise = trial.suggest_categorical('noise', [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009])       # - Noise Set
    look_back = trial.suggest_int('look_back', 10, 500)

    # convert into input/output
    X, Y = create_multivariate_dataset(dataset, look_back)

    # Reshape X in 2D Dimention
    X = np.reshape(X, (X.shape[0], -1))

    test_size = int(len(X) * 0.10)

    # Divisez manuellement X et Y en ensembles d'entraînement et de test
    trainX, trainY, testX, testY = X[:-test_size], Y[:-test_size], X[-test_size:], Y[-test_size:]
    print(trainX.shape, trainY.shape)

    model = ESN(n_inputs = trainX.shape[1],
            n_outputs = Y.shape[1],
            n_reservoir = n_reservoir,
            sparsity=sparsity,
            random_state=1234,
            spectral_radius=spectral_radius,
            noise = noise,
            silent = False,
    )

    pred_train = model.fit(trainX, trainY)

    # make predictions
    testPredict = model.predict(testX)
    testPredict = np.array(testPredict)

    node1_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 0], testPredict[:len(testY), 0]))
    node2_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 1], testPredict[:len(testY), 1]))
    node3_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 2], testPredict[:len(testY), 2]))

    print(node1_delay_RMSE, node2_delay_RMSE, node3_delay_RMSE)

    testRMSE = np.sqrt(mean_squared_error(testY, testPredict[:len(testY), :]))

    return node1_delay_RMSE


# Create the Optuna study
study = optuna.create_study(direction='minimize')

# Record the starting time to generate predictions
start_time = time.time()

# Run the optimization
study.optimize(objective, n_trials=100)

# Record the ending time
end_time = time.time()
elapsed_time = end_time - start_time

print('done')
print("ESN HyperParameters Tuning Elapsed Time : %.5f" % (elapsed_time), "seconds")

# Print the best parameters and corresponding loss
best_params = study.best_params
best_loss = study.best_value
print("Best Parameters: ", best_params)
print("Best Loss: ", best_loss)

[I 2023-09-07 16:24:32,693] A new study created in memory with name: no-name-2411cd83-b5a3-4bdf-892c-cc8af1f201e4


Window Size1:  10
Window Size2:  3
Window Size3:  10
(17584, 1362) (17584, 3)
harvesting states...
fitting...
training error:
0.32712300224153756


[I 2023-09-07 16:25:31,401] Trial 0 finished with value: 0.02015482128584641 and parameters: {'n_reservoir': 473, 'sparsity': 0.01, 'spectral_radius': 0.4, 'noise': 0.004, 'look_back': 454}. Best is trial 0 with value: 0.02015482128584641.


0.02015482128584641 0.6106946444906072 0.22389371466747313
(17550, 1473) (17550, 3)
harvesting states...
fitting...
training error:
0.3252091040046546


[I 2023-09-07 16:26:13,235] Trial 1 finished with value: 0.02139679843707324 and parameters: {'n_reservoir': 496, 'sparsity': 0.2, 'spectral_radius': 0.2, 'noise': 0.003, 'look_back': 491}. Best is trial 0 with value: 0.02015482128584641.


...

training error:
0.34353058012645254
0.016371514534480356 0.5842990551607987 0.18287782773982036
(17859, 444) (17859, 3)
harvesting states...
fitting...
training error:
0.3424618705204632


[I 2023-09-07 16:42:32,943] Trial 98 finished with value: 0.016050688768833584 and parameters: {'n_reservoir': 59, 'sparsity': 0.3, 'spectral_radius': 0.3, 'noise': 0.005, 'look_back': 148}. Best is trial 53 with value: 0.01599731029336741.


0.016050688768833584 0.5798400125233372 0.18004876754242433
(17841, 504) (17841, 3)
harvesting states...
fitting...


[I 2023-09-07 16:42:38,407] Trial 99 finished with value: 0.0161555600793235 and parameters: {'n_reservoir': 134, 'sparsity': 0.4, 'spectral_radius': 10.0, 'noise': 0.007, 'look_back': 168}. Best is trial 53 with value: 0.01599731029336741.


training error:
0.34080172804998077
0.0161555600793235 0.5816418352371637 0.18035783427855
done
ESN HyperParameters Tuning Elapsed Time : 1085.71912 seconds
Best Parameters:  {'n_reservoir': 11, 'sparsity': 0.4, 'spectral_radius': 0.5, 'noise': 0.005, 'look_back': 216}
Best Loss:  0.01599731029336741


## Prédiction multivariée de chaque caractéristique pour les 3 noeuds

### Delay

In [ ]:
node1_delay = df[['node1_delay']]
node2_delay = df[['node2_delay']]
node3_delay = df[['node3_delay']]

node1_delay_dataset = node1_delay.values
node2_delay_dataset = node2_delay.values
node3_delay_dataset = node3_delay.values

window_size1, slide_size1 = smooth_ASAP(node1_delay_dataset, resolution=50)
window_size2, slide_size2 = smooth_ASAP(node2_delay_dataset, resolution=50)
window_size3, slide_size3 = smooth_ASAP(node3_delay_dataset, resolution=50)

print("Window Size1: ", window_size1)
print("Window Size2: ", window_size2)
print("Window Size3: ", window_size3)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size1)
denoised_node2_delay_dataset = moving_average(node2_delay_dataset, window_size2)
denoised_node3_delay_dataset = moving_average(node3_delay_dataset, window_size3)

# Déterminez la longueur minimale des trois séries
min_length = min(len(denoised_node1_delay_dataset), len(denoised_node2_delay_dataset), len(denoised_node3_delay_dataset))

# convert to [rows, columns] structure
denoised_node1_delay_dataset = denoised_node1_delay_dataset.reshape((len(denoised_node1_delay_dataset), 1))
denoised_node2_delay_dataset = denoised_node2_delay_dataset.reshape((len(denoised_node2_delay_dataset), 1))
denoised_node3_delay_dataset = denoised_node3_delay_dataset.reshape((len(denoised_node3_delay_dataset), 1))

# horizontally stack columns
dataset = np.hstack((denoised_node1_delay_dataset[:min_length], denoised_node2_delay_dataset[:min_length], denoised_node3_delay_dataset[:min_length]))

# Define the objective function for Optuna optimization
def objective(trial):
    # Sample hyperparameters to search

    n_reservoir = trial.suggest_int('n_reservoir', 10, 1000)   # -
    sparsity = trial.suggest_categorical('sparsity', [0.01, 0.1, 0.2, 0.3, 0.4, 0.5])   # -
    spectral_radius = trial.suggest_categorical('spectral_radius', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.25, 10.0])   # - spectral radius of W
    noise = trial.suggest_categorical('noise', [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009])       # - Noise Set
    look_back = trial.suggest_int('look_back', 10, 500)

    # convert into input/output
    X, Y = create_multivariate_dataset(dataset, look_back)

    # Reshape X in 2D Dimention
    X = np.reshape(X, (X.shape[0], -1))

    test_size = int(len(X) * 0.10)

    # Divisez manuellement X et Y en ensembles d'entraînement et de test
    trainX, trainY, testX, testY = X[:-test_size], Y[:-test_size], X[-test_size:], Y[-test_size:]
    print(trainX.shape, trainY.shape)

    model = ESN(n_inputs = trainX.shape[1],
            n_outputs = Y.shape[1],
            n_reservoir = n_reservoir,
            sparsity=sparsity,
            random_state=1234,
            spectral_radius=spectral_radius,
            noise = noise,
            silent = False,
    )

    pred_train = model.fit(trainX, trainY)

    # make predictions
    testPredict = model.predict(testX)
    testPredict = np.array(testPredict)

    node1_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 0], testPredict[:len(testY), 0]))
    node2_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 1], testPredict[:len(testY), 1]))
    node3_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 2], testPredict[:len(testY), 2]))

    print(node1_delay_RMSE, node2_delay_RMSE, node3_delay_RMSE)

    testRMSE = np.sqrt(mean_squared_error(testY, testPredict[:len(testY), :]))

    return node1_delay_RMSE


# Create the Optuna study
study = optuna.create_study(direction='minimize')

# Record the starting time to generate predictions
start_time = time.time()

# Run the optimization
study.optimize(objective, n_trials=100)

# Record the ending time
end_time = time.time()
elapsed_time = end_time - start_time

print('done')
print("ESN HyperParameters Tuning Elapsed Time : %.5f" % (elapsed_time), "seconds")

# Print the best parameters and corresponding loss
best_params = study.best_params
best_loss = study.best_value
print("Best Parameters: ", best_params)
print("Best Loss: ", best_loss)

[I 2023-09-08 10:29:05,722] A new study created in memory with name: no-name-ca8c5252-d18e-4659-aee9-84395916bf87


Window Size1:  10
Window Size2:  9
Window Size3:  10
(17744, 828) (17744, 3)
harvesting states...
fitting...
training error:
0.010404282610432261


[I 2023-09-08 10:29:42,449] Trial 0 finished with value: 0.016274226561259746 and parameters: {'n_reservoir': 620, 'sparsity': 0.3, 'spectral_radius': 0.8, 'noise': 0.008, 'look_back': 276}. Best is trial 0 with value: 0.016274226561259746.


0.016274226561259746 0.006367366716982629 0.009309632738351737
(17965, 90) (17965, 3)
harvesting states...
fitting...
training error:
0.011714189206990244


[I 2023-09-08 10:29:46,564] Trial 1 finished with value: 0.017283853228895397 and parameters: {'n_reservoir': 209, 'sparsity': 0.3, 'spectral_radius': 1.25, 'noise': 0.009, 'look_back': 30}. Best is trial 0 with value: 0.016274226561259746.


...

training error:
0.010429262731948287
0.016053689491103612 0.006197467264505062 0.009074148426647994
(17661, 1104) (17661, 3)
harvesting states...
fitting...
training error:
0.010404965888148583


[I 2023-09-08 10:56:45,200] Trial 98 finished with value: 0.016056440575646903 and parameters: {'n_reservoir': 172, 'sparsity': 0.4, 'spectral_radius': 1, 'noise': 0.002, 'look_back': 368}. Best is trial 21 with value: 0.015917901578099927.


0.016056440575646903 0.0062112420986663325 0.009087162233853023
(17672, 1068) (17672, 3)
harvesting states...
fitting...
training error:
0.010441437018068097


[I 2023-09-08 10:56:59,155] Trial 99 finished with value: 0.0160414815250882 and parameters: {'n_reservoir': 119, 'sparsity': 0.01, 'spectral_radius': 10.0, 'noise': 0.006, 'look_back': 356}. Best is trial 21 with value: 0.015917901578099927.


0.0160414815250882 0.0061867531830127375 0.009099159252316767
done
ESN HyperParameters Tuning Elapsed Time : 1673.43737 seconds
Best Parameters:  {'n_reservoir': 12, 'sparsity': 0.2, 'spectral_radius': 0.4, 'noise': 0.008, 'look_back': 348}
Best Loss:  0.015917901578099927


In [ ]:
# Best Params
n_reservoir = 12
sparsity = 0.2
spectral_radius = 0.4
noise = 0.008
look_back = 348

In [ ]:
node1_delay = df[['node1_delay']]
node2_delay = df[['node2_delay']]
node3_delay = df[['node3_delay']]

node1_delay_dataset = node1_delay.values
node2_delay_dataset = node2_delay.values
node3_delay_dataset = node3_delay.values

window_size1, slide_size1 = smooth_ASAP(node1_delay_dataset, resolution=50)
window_size2, slide_size2 = smooth_ASAP(node2_delay_dataset, resolution=50)
window_size3, slide_size3 = smooth_ASAP(node3_delay_dataset, resolution=50)

print("Window Size1: ", window_size1)
print("Window Size2: ", window_size2)
print("Window Size3: ", window_size3)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size1)
denoised_node2_delay_dataset = moving_average(node2_delay_dataset, window_size2)
denoised_node3_delay_dataset = moving_average(node3_delay_dataset, window_size3)

# Déterminez la longueur minimale des trois séries
min_length = min(len(denoised_node1_delay_dataset), len(denoised_node2_delay_dataset), len(denoised_node3_delay_dataset))

# convert to [rows, columns] structure
denoised_node1_delay_dataset = denoised_node1_delay_dataset.reshape((len(denoised_node1_delay_dataset), 1))
denoised_node2_delay_dataset = denoised_node2_delay_dataset.reshape((len(denoised_node2_delay_dataset), 1))
denoised_node3_delay_dataset = denoised_node3_delay_dataset.reshape((len(denoised_node3_delay_dataset), 1))

# horizontally stack columns
dataset = np.hstack((denoised_node1_delay_dataset[:min_length], denoised_node2_delay_dataset[:min_length], denoised_node3_delay_dataset[:min_length]))

# convert into input/output
X, Y = create_multivariate_dataset(dataset, look_back)

# Reshape X in 2D Dimention
X = np.reshape(X, (X.shape[0], -1))

test_size = int(len(X) * 0.10)

# Divisez manuellement X et Y en ensembles d'entraînement et de test
trainX, trainY, testX, testY = X[:-test_size], Y[:-test_size], X[-test_size:], Y[-test_size:]
print(trainX.shape, trainY.shape)

model = ESN(n_inputs = trainX.shape[1],
        n_outputs = Y.shape[1],
        n_reservoir = n_reservoir,
        sparsity=sparsity,
        random_state=1234,
        spectral_radius=spectral_radius,
        noise = noise,
        silent = False,
)

# Record the starting time to training the model
training_start_time = time.time()

pred_train = model.fit(trainX, trainY)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time

# Record the starting time to generate predictions
predictions_start_time = time.time()

# make predictions
testPredict = model.predict(testX)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time

testPredict = np.array(testPredict)

node1_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 0], testPredict[:len(testY), 0]))
node2_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 1], testPredict[:len(testY), 1]))
node3_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 2], testPredict[:len(testY), 2]))

node1_delay_MAE = mean_absolute_error(testY[:, 0], testPredict[:len(testY), 0])
node2_delay_MAE = mean_absolute_error(testY[:, 1], testPredict[:len(testY), 1])
node3_delay_MAE = mean_absolute_error(testY[:, 2], testPredict[:len(testY), 2])



testRMSE = np.sqrt(mean_squared_error(testY, testPredict[:len(testY), :]))
testMAE = mean_absolute_error(testY, testPredict[:len(testY), :])

print("Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

print('Node1 Delay Test RMSE : %.5f' % (node1_delay_RMSE))
print('Node2 Delay Test RMSE : %.5f' % (node2_delay_RMSE))
print('Node3 Delay Test RMSE : %.5f' % (node3_delay_RMSE))
print('Global Test RMSE : %.5f' % (testRMSE))

print('Node1 Delay Test MAE : %.5f' % (node1_delay_MAE))
print('Node2 Delay Test MAE : %.5f' % (node2_delay_MAE))
print('Node3 Delay Test MAE : %.5f' % (node3_delay_MAE))
print('Global Test MAE : %.5f' % (testMAE))

print("Model Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size1:  10
Window Size2:  9
Window Size3:  10
(17679, 1044) (17679, 3)
harvesting states...
fitting...
training error:
0.010484499422695682
Model Training Elapsed Time : 11.93710 seconds
Node1 Delay Test RMSE : 0.01592
Node2 Delay Test RMSE : 0.00617
Node3 Delay Test RMSE : 0.00905
Global Test RMSE : 0.01115
Node1 Delay Test MAE : 0.01359
Node2 Delay Test MAE : 0.00526
Node3 Delay Test MAE : 0.00775
Global Test MAE : 0.00887
Model Elapsed Time To generate Predictions : 0.57046 seconds


### CPU

In [ ]:
node1_delay = df[['node1_cpu']]
node2_delay = df[['node2_cpu']]
node3_delay = df[['node3_cpu']]

node1_delay_dataset = node1_delay.values
node2_delay_dataset = node2_delay.values
node3_delay_dataset = node3_delay.values

window_size1, slide_size1 = smooth_ASAP(node1_delay_dataset, resolution=50)
window_size2, slide_size2 = smooth_ASAP(node2_delay_dataset, resolution=100)
window_size3, slide_size3 = smooth_ASAP(node3_delay_dataset, resolution=50)

print("Window Size1: ", window_size1)
print("Window Size2: ", window_size2)
print("Window Size3: ", window_size3)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size1)
denoised_node2_delay_dataset = moving_average(node2_delay_dataset, window_size2)
denoised_node3_delay_dataset = moving_average(node3_delay_dataset, window_size3)

# Déterminez la longueur minimale des trois séries
min_length = min(len(denoised_node1_delay_dataset), len(denoised_node2_delay_dataset), len(denoised_node3_delay_dataset))

# convert to [rows, columns] structure
denoised_node1_delay_dataset = denoised_node1_delay_dataset.reshape((len(denoised_node1_delay_dataset), 1))
denoised_node2_delay_dataset = denoised_node2_delay_dataset.reshape((len(denoised_node2_delay_dataset), 1))
denoised_node3_delay_dataset = denoised_node3_delay_dataset.reshape((len(denoised_node3_delay_dataset), 1))

# horizontally stack columns
dataset = np.hstack((denoised_node1_delay_dataset[:min_length], denoised_node2_delay_dataset[:min_length], denoised_node3_delay_dataset[:min_length]))

# Define the objective function for Optuna optimization
def objective(trial):
    # Sample hyperparameters to search

    n_reservoir = trial.suggest_int('n_reservoir', 10, 1000)   # -
    sparsity = trial.suggest_categorical('sparsity', [0.01, 0.1, 0.2, 0.3, 0.4, 0.5])   # -
    spectral_radius = trial.suggest_categorical('spectral_radius', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.25, 10.0])   # - spectral radius of W
    noise = trial.suggest_categorical('noise', [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009])       # - Noise Set
    look_back = trial.suggest_int('look_back', 10, 500)

    # convert into input/output
    X, Y = create_multivariate_dataset(dataset, look_back)

    # Reshape X in 2D Dimention
    X = np.reshape(X, (X.shape[0], -1))

    test_size = int(len(X) * 0.10)

    # Divisez manuellement X et Y en ensembles d'entraînement et de test
    trainX, trainY, testX, testY = X[:-test_size], Y[:-test_size], X[-test_size:], Y[-test_size:]
    print(trainX.shape, trainY.shape)

    model = ESN(n_inputs = trainX.shape[1],
            n_outputs = Y.shape[1],
            n_reservoir = n_reservoir,
            sparsity=sparsity,
            random_state=1234,
            spectral_radius=spectral_radius,
            noise = noise,
            silent = False,
    )

    pred_train = model.fit(trainX, trainY)

    # make predictions
    testPredict = model.predict(testX)
    testPredict = np.array(testPredict)

    node1_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 0], testPredict[:len(testY), 0]))
    node2_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 1], testPredict[:len(testY), 1]))
    node3_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 2], testPredict[:len(testY), 2]))

    print(node1_delay_RMSE, node2_delay_RMSE, node3_delay_RMSE)

    testRMSE = np.sqrt(mean_squared_error(testY, testPredict[:len(testY), :]))

    return node1_delay_RMSE


# Create the Optuna study
study = optuna.create_study(direction='minimize')

# Record the starting time to generate predictions
start_time = time.time()

# Run the optimization
study.optimize(objective, n_trials=100)

# Record the ending time
end_time = time.time()
elapsed_time = end_time - start_time

print('done')
print("ESN HyperParameters Tuning Elapsed Time : %.5f" % (elapsed_time), "seconds")

# Print the best parameters and corresponding loss
best_params = study.best_params
best_loss = study.best_value
print("Best Parameters: ", best_params)
print("Best Loss: ", best_loss)

[I 2023-09-08 10:56:59,371] A new study created in memory with name: no-name-243f5595-1cc2-4e91-8ac7-53f1a6132fbe


Window Size1:  3
Window Size2:  2
Window Size3:  10
(17731, 870) (17731, 3)
harvesting states...
fitting...
training error:
0.08454301795622945


[I 2023-09-08 10:57:33,286] Trial 0 finished with value: 0.08969051903395578 and parameters: {'n_reservoir': 759, 'sparsity': 0.01, 'spectral_radius': 1, 'noise': 0.005, 'look_back': 290}. Best is trial 0 with value: 0.08969051903395578.


0.08969051903395578 0.13150950459571992 0.026465024031672123
(17805, 624) (17805, 3)
harvesting states...
fitting...
training error:
0.08534868874473432


[I 2023-09-08 10:58:00,207] Trial 1 finished with value: 0.09160413684730703 and parameters: {'n_reservoir': 736, 'sparsity': 0.1, 'spectral_radius': 1, 'noise': 0.002, 'look_back': 208}. Best is trial 0 with value: 0.08969051903395578.


...

training error:
0.08841622600326905
0.08691063873216882 0.12669917461991725 0.026654858928685167
(17589, 1344) (17589, 3)
harvesting states...
fitting...


[I 2023-09-08 11:11:39,542] Trial 98 finished with value: 0.08919829336817833 and parameters: {'n_reservoir': 32, 'sparsity': 0.3, 'spectral_radius': 10.0, 'noise': 0.001, 'look_back': 448}. Best is trial 71 with value: 0.08659595287099897.


training error:
0.08506238763739812
0.08919829336817833 0.12881231928050849 0.026350332351533362
(17836, 522) (17836, 3)
harvesting states...
fitting...


[I 2023-09-08 11:11:42,655] Trial 99 finished with value: 0.08743579737273183 and parameters: {'n_reservoir': 10, 'sparsity': 0.2, 'spectral_radius': 0.4, 'noise': 0.009, 'look_back': 174}. Best is trial 71 with value: 0.08659595287099897.


training error:
0.0875843790606434
0.08743579737273183 0.12697660132016123 0.026082426572525783
done
ESN HyperParameters Tuning Elapsed Time : 883.28114 seconds
Best Parameters:  {'n_reservoir': 83, 'sparsity': 0.2, 'spectral_radius': 0.7, 'noise': 0.008, 'look_back': 79}
Best Loss:  0.08659595287099897


In [ ]:
# Best Params
n_reservoir = 83
sparsity = 0.2
spectral_radius = 0.7
noise = 0.008
look_back = 79

In [ ]:
node1_delay = df[['node1_cpu']]
node2_delay = df[['node2_cpu']]
node3_delay = df[['node3_cpu']]

node1_delay_dataset = node1_delay.values
node2_delay_dataset = node2_delay.values
node3_delay_dataset = node3_delay.values

window_size1, slide_size1 = smooth_ASAP(node1_delay_dataset, resolution=50)
window_size2, slide_size2 = smooth_ASAP(node2_delay_dataset, resolution=100)
window_size3, slide_size3 = smooth_ASAP(node3_delay_dataset, resolution=50)

print("Window Size1: ", window_size1)
print("Window Size2: ", window_size2)
print("Window Size3: ", window_size3)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size1)
denoised_node2_delay_dataset = moving_average(node2_delay_dataset, window_size2)
denoised_node3_delay_dataset = moving_average(node3_delay_dataset, window_size3)

# Déterminez la longueur minimale des trois séries
min_length = min(len(denoised_node1_delay_dataset), len(denoised_node2_delay_dataset), len(denoised_node3_delay_dataset))

# convert to [rows, columns] structure
denoised_node1_delay_dataset = denoised_node1_delay_dataset.reshape((len(denoised_node1_delay_dataset), 1))
denoised_node2_delay_dataset = denoised_node2_delay_dataset.reshape((len(denoised_node2_delay_dataset), 1))
denoised_node3_delay_dataset = denoised_node3_delay_dataset.reshape((len(denoised_node3_delay_dataset), 1))

# horizontally stack columns
dataset = np.hstack((denoised_node1_delay_dataset[:min_length], denoised_node2_delay_dataset[:min_length], denoised_node3_delay_dataset[:min_length]))

# convert into input/output
X, Y = create_multivariate_dataset(dataset, look_back)

# Reshape X in 2D Dimention
X = np.reshape(X, (X.shape[0], -1))

test_size = int(len(X) * 0.10)

# Divisez manuellement X et Y en ensembles d'entraînement et de test
trainX, trainY, testX, testY = X[:-test_size], Y[:-test_size], X[-test_size:], Y[-test_size:]
print(trainX.shape, trainY.shape)

model = ESN(n_inputs = trainX.shape[1],
        n_outputs = Y.shape[1],
        n_reservoir = n_reservoir,
        sparsity=sparsity,
        random_state=1234,
        spectral_radius=spectral_radius,
        noise = noise,
        silent = False,
)

# Record the starting time to training the model
training_start_time = time.time()

pred_train = model.fit(trainX, trainY)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time

# Record the starting time to generate predictions
predictions_start_time = time.time()

# make predictions
testPredict = model.predict(testX)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time

testPredict = np.array(testPredict)

node1_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 0], testPredict[:len(testY), 0]))
node2_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 1], testPredict[:len(testY), 1]))
node3_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 2], testPredict[:len(testY), 2]))

node1_delay_MAE = mean_absolute_error(testY[:, 0], testPredict[:len(testY), 0])
node2_delay_MAE = mean_absolute_error(testY[:, 1], testPredict[:len(testY), 1])
node3_delay_MAE = mean_absolute_error(testY[:, 2], testPredict[:len(testY), 2])



testRMSE = np.sqrt(mean_squared_error(testY, testPredict[:len(testY), :]))
testMAE = mean_absolute_error(testY, testPredict[:len(testY), :])

print("Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

print('Node1 CPU Test RMSE : %.5f' % (node1_delay_RMSE))
print('Node2 CPU Test RMSE : %.5f' % (node2_delay_RMSE))
print('Node3 CPU Test RMSE : %.5f' % (node3_delay_RMSE))
print('Global Test RMSE : %.5f' % (testRMSE))

print('Node1 CPU Test MAE : %.5f' % (node1_delay_MAE))
print('Node2 CPU Test MAE : %.5f' % (node2_delay_MAE))
print('Node3 CPU Test MAE : %.5f' % (node3_delay_MAE))
print('Global Test MAE : %.5f' % (testMAE))

print("Model Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size1:  3
Window Size2:  2
Window Size3:  10
(17921, 237) (17921, 3)
harvesting states...
fitting...
training error:
0.08854960003297246
Model Training Elapsed Time : 1.93479 seconds
Node1 CPU Test RMSE : 0.08660
Node2 CPU Test RMSE : 0.12643
Node3 CPU Test RMSE : 0.02699
Global Test RMSE : 0.08984
Node1 CPU Test MAE : 0.07498
Node2 CPU Test MAE : 0.10895
Node3 CPU Test MAE : 0.02300
Global Test MAE : 0.06898
Model Elapsed Time To generate Predictions : 0.09823 seconds


## Ram

In [ ]:
node1_delay = df[['node1_ram']]
node2_delay = df[['node2_ram']]
node3_delay = df[['node3_ram']]

node1_delay_dataset = node1_delay.values
node2_delay_dataset = node2_delay.values
node3_delay_dataset = node3_delay.values

window_size1, slide_size1 = smooth_ASAP(node1_delay_dataset, resolution=50)
window_size2, slide_size2 = smooth_ASAP(node2_delay_dataset, resolution=50)
window_size3, slide_size3 = smooth_ASAP(node3_delay_dataset, resolution=50)

print("Window Size1: ", window_size1)
print("Window Size2: ", window_size2)
print("Window Size3: ", window_size3)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size1)
denoised_node2_delay_dataset = moving_average(node2_delay_dataset, window_size2)
denoised_node3_delay_dataset = moving_average(node3_delay_dataset, window_size3)

# Déterminez la longueur minimale des trois séries
min_length = min(len(denoised_node1_delay_dataset), len(denoised_node2_delay_dataset), len(denoised_node3_delay_dataset))

# convert to [rows, columns] structure
denoised_node1_delay_dataset = denoised_node1_delay_dataset.reshape((len(denoised_node1_delay_dataset), 1))
denoised_node2_delay_dataset = denoised_node2_delay_dataset.reshape((len(denoised_node2_delay_dataset), 1))
denoised_node3_delay_dataset = denoised_node3_delay_dataset.reshape((len(denoised_node3_delay_dataset), 1))

# horizontally stack columns
dataset = np.hstack((denoised_node1_delay_dataset[:min_length], denoised_node2_delay_dataset[:min_length], denoised_node3_delay_dataset[:min_length]))

# Define the objective function for Optuna optimization
def objective(trial):
    # Sample hyperparameters to search

    n_reservoir = trial.suggest_int('n_reservoir', 10, 1000)   # -
    sparsity = trial.suggest_categorical('sparsity', [0.01, 0.1, 0.2, 0.3, 0.4, 0.5])   # -
    spectral_radius = trial.suggest_categorical('spectral_radius', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.25, 10.0])   # - spectral radius of W
    noise = trial.suggest_categorical('noise', [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009])       # - Noise Set
    look_back = trial.suggest_int('look_back', 10, 500)

    # convert into input/output
    X, Y = create_multivariate_dataset(dataset, look_back)

    # Reshape X in 2D Dimention
    X = np.reshape(X, (X.shape[0], -1))

    test_size = int(len(X) * 0.10)

    # Divisez manuellement X et Y en ensembles d'entraînement et de test
    trainX, trainY, testX, testY = X[:-test_size], Y[:-test_size], X[-test_size:], Y[-test_size:]
    print(trainX.shape, trainY.shape)

    model = ESN(n_inputs = trainX.shape[1],
            n_outputs = Y.shape[1],
            n_reservoir = n_reservoir,
            sparsity=sparsity,
            random_state=1234,
            spectral_radius=spectral_radius,
            noise = noise,
            silent = False,
    )

    pred_train = model.fit(trainX, trainY)

    # make predictions
    testPredict = model.predict(testX)
    testPredict = np.array(testPredict)

    node1_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 0], testPredict[:len(testY), 0]))
    node2_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 1], testPredict[:len(testY), 1]))
    node3_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 2], testPredict[:len(testY), 2]))

    print(node1_delay_RMSE, node2_delay_RMSE, node3_delay_RMSE)

    testRMSE = np.sqrt(mean_squared_error(testY, testPredict[:len(testY), :]))

    return node1_delay_RMSE


# Create the Optuna study
study = optuna.create_study(direction='minimize')

# Record the starting time to generate predictions
start_time = time.time()

# Run the optimization
study.optimize(objective, n_trials=100)

# Record the ending time
end_time = time.time()
elapsed_time = end_time - start_time

print('done')
print("ESN HyperParameters Tuning Elapsed Time : %.5f" % (elapsed_time), "seconds")

# Print the best parameters and corresponding loss
best_params = study.best_params
best_loss = study.best_value
print("Best Parameters: ", best_params)
print("Best Loss: ", best_loss)

[I 2023-09-08 11:11:42,728] A new study created in memory with name: no-name-1c7d861b-9833-406b-9460-1c4d4ef7f035


Window Size1:  10
Window Size2:  6
Window Size3:  9
(17968, 81) (17968, 3)
harvesting states...
fitting...
training error:
0.02082643486160828


[I 2023-09-08 11:11:58,754] Trial 0 finished with value: 0.017558834713804747 and parameters: {'n_reservoir': 745, 'sparsity': 0.4, 'spectral_radius': 0.2, 'noise': 0.005, 'look_back': 27}. Best is trial 0 with value: 0.017558834713804747.


0.017558834713804747 0.028626855141070114 0.019724702861499105
(17876, 387) (17876, 3)
harvesting states...
fitting...
training error:
0.019008551875166448


[I 2023-09-08 11:12:32,884] Trial 1 finished with value: 0.016039097816780104 and parameters: {'n_reservoir': 998, 'sparsity': 0.01, 'spectral_radius': 0.6, 'noise': 0.004, 'look_back': 129}. Best is trial 1 with value: 0.016039097816780104.


...

0.015604590188648279 0.026215254910537653 0.017330610999092323
(17694, 996) (17694, 3)
harvesting states...
fitting...


[I 2023-09-08 11:35:22,151] Trial 98 finished with value: 0.015595312818410815 and parameters: {'n_reservoir': 10, 'sparsity': 0.1, 'spectral_radius': 0.1, 'noise': 0.009, 'look_back': 332}. Best is trial 48 with value: 0.015555293818041107.


training error:
0.018977238052926518
0.015595312818410815 0.026257990951774304 0.01728161312302115
(17768, 747) (17768, 3)
harvesting states...
fitting...


[I 2023-09-08 11:35:29,899] Trial 99 finished with value: 0.015608635876453617 and parameters: {'n_reservoir': 60, 'sparsity': 0.3, 'spectral_radius': 1, 'noise': 0.005, 'look_back': 249}. Best is trial 48 with value: 0.015555293818041107.


training error:
0.019143058793235412
0.015608635876453617 0.026189451163367225 0.017252785811582818
done
ESN HyperParameters Tuning Elapsed Time : 1427.17096 seconds
Best Parameters:  {'n_reservoir': 11, 'sparsity': 0.5, 'spectral_radius': 0.3, 'noise': 0.009, 'look_back': 264}
Best Loss:  0.015555293818041107


In [ ]:
# Best Params
n_reservoir = 11
sparsity = 0.5
spectral_radius = 0.3
noise = 0.009
look_back = 264

In [ ]:
node1_delay = df[['node1_ram']]
node2_delay = df[['node2_ram']]
node3_delay = df[['node3_ram']]

node1_delay_dataset = node1_delay.values
node2_delay_dataset = node2_delay.values
node3_delay_dataset = node3_delay.values

window_size1, slide_size1 = smooth_ASAP(node1_delay_dataset, resolution=50)
window_size2, slide_size2 = smooth_ASAP(node2_delay_dataset, resolution=50)
window_size3, slide_size3 = smooth_ASAP(node3_delay_dataset, resolution=50)

print("Window Size1: ", window_size1)
print("Window Size2: ", window_size2)
print("Window Size3: ", window_size3)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size1)
denoised_node2_delay_dataset = moving_average(node2_delay_dataset, window_size2)
denoised_node3_delay_dataset = moving_average(node3_delay_dataset, window_size3)

# Déterminez la longueur minimale des trois séries
min_length = min(len(denoised_node1_delay_dataset), len(denoised_node2_delay_dataset), len(denoised_node3_delay_dataset))

# convert to [rows, columns] structure
denoised_node1_delay_dataset = denoised_node1_delay_dataset.reshape((len(denoised_node1_delay_dataset), 1))
denoised_node2_delay_dataset = denoised_node2_delay_dataset.reshape((len(denoised_node2_delay_dataset), 1))
denoised_node3_delay_dataset = denoised_node3_delay_dataset.reshape((len(denoised_node3_delay_dataset), 1))

# horizontally stack columns
dataset = np.hstack((denoised_node1_delay_dataset[:min_length], denoised_node2_delay_dataset[:min_length], denoised_node3_delay_dataset[:min_length]))

# convert into input/output
X, Y = create_multivariate_dataset(dataset, look_back)

# Reshape X in 2D Dimention
X = np.reshape(X, (X.shape[0], -1))

test_size = int(len(X) * 0.10)

# Divisez manuellement X et Y en ensembles d'entraînement et de test
trainX, trainY, testX, testY = X[:-test_size], Y[:-test_size], X[-test_size:], Y[-test_size:]
print(trainX.shape, trainY.shape)

model = ESN(n_inputs = trainX.shape[1],
        n_outputs = Y.shape[1],
        n_reservoir = n_reservoir,
        sparsity=sparsity,
        random_state=1234,
        spectral_radius=spectral_radius,
        noise = noise,
        silent = False,
)

# Record the starting time to training the model
training_start_time = time.time()

pred_train = model.fit(trainX, trainY)

# Record the ending time of training the model
training_end_time = time.time()
training_elapsed_time = training_end_time - training_start_time

# Record the starting time to generate predictions
predictions_start_time = time.time()

# make predictions
testPredict = model.predict(testX)

# Record the ending time of generating predictions
predictions_end_time = time.time()
predictions_elapsed_time = predictions_end_time - predictions_start_time

testPredict = np.array(testPredict)

node1_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 0], testPredict[:len(testY), 0]))
node2_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 1], testPredict[:len(testY), 1]))
node3_delay_RMSE = np.sqrt(mean_squared_error(testY[:, 2], testPredict[:len(testY), 2]))

node1_delay_MAE = mean_absolute_error(testY[:, 0], testPredict[:len(testY), 0])
node2_delay_MAE = mean_absolute_error(testY[:, 1], testPredict[:len(testY), 1])
node3_delay_MAE = mean_absolute_error(testY[:, 2], testPredict[:len(testY), 2])



testRMSE = np.sqrt(mean_squared_error(testY, testPredict[:len(testY), :]))
testMAE = mean_absolute_error(testY, testPredict[:len(testY), :])

print("Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

print('Node1 RAM Test RMSE : %.5f' % (node1_delay_RMSE))
print('Node2 RAM Test RMSE : %.5f' % (node2_delay_RMSE))
print('Node3 RAM Test RMSE : %.5f' % (node3_delay_RMSE))
print('Global Test RMSE : %.5f' % (testRMSE))

print('Node1 RAM Test MAE : %.5f' % (node1_delay_MAE))
print('Node2 RAM Test MAE : %.5f' % (node2_delay_MAE))
print('Node3 RAM Test MAE : %.5f' % (node3_delay_MAE))
print('Global Test MAE : %.5f' % (testMAE))

print("Model Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size1:  10
Window Size2:  6
Window Size3:  9
(17755, 792) (17755, 3)
harvesting states...
fitting...
training error:
0.01913093683003193
Model Training Elapsed Time : 10.71825 seconds
Node1 RAM Test RMSE : 0.01556
Node2 RAM Test RMSE : 0.02619
Node3 RAM Test RMSE : 0.01725
Global Test RMSE : 0.02021
Node1 RAM Test MAE : 0.01310
Node2 RAM Test MAE : 0.02236
Node3 RAM Test MAE : 0.01456
Global Test MAE : 0.01667
Model Elapsed Time To generate Predictions : 0.14634 seconds


In [ ]:
import numpy as np

ESN = [2.83301, 2.20503, 1.99874, 2.04051, 2.14621, 2.04692, 3.34907, 1.36598, 2.11508]

m = np.mean(ESN)
s = np.std(ESN)

print(m, s)

2.2333944444444445 0.527691238701638


In [ ]:
LSTM = [275.31462, 169.30349, 206.01040, 95.26364, 207.64390, 164.26032, 205.38146, 209.73208, 205.57009]
GRU = [205.47089, 156.79970, 156.55630, 80.54434, 160.47253, 152.58536, 152.14188, 172.11567, 153.42972]
RNN = [190.75704, 206.74629, 197.93322, 104.34716, 222.46698, 189.28047, 189.27849, 84.78259, 203.39467]

LSTM_m = np.mean(LSTM)
GRU_m = np.mean(GRU)
RNN_m = np.mean(RNN)

print(LSTM_m, GRU_m, RNN_m)

193.16444444444443 154.45737666666668 176.5541011111111


In [ ]:
ARIMA_1 = [262.20982, 296.69617, 237.76282, 283.59772, 285.87563, 289.59900, 315.98457, 331.13528, 295.24045]
ARIMA_2 = [0.13110, 0.14835, 0.11888, 0.14180, 0.14294, 0.14480, 0.15799, 0.16557, 0.14762]

ARIMA_1_m = np.mean(ARIMA_1)
ARIMA_2_m = np.mean(ARIMA_2)

print(ARIMA_1_m, ARIMA_2_m)

288.67794 0.1443388888888889


In [ ]:
print(np.std(ARIMA_1))

25.783407801492128


# DirRec Strategy

In [ ]:
def create_multistep_dataset(data, n_input, n_out=1):
    X, y = list(), list()
    in_start = 0
    # step over the entire history one time step at a time
    for _ in range(len(data)):
        # define the end of the input sequence
        in_end = in_start + n_input
        out_end = in_end + n_out
        # ensure we have enough data for this instance
        if out_end <= len(data):
            x_input = data[in_start:in_end]
            X.append(x_input)
            y.append(data[in_end:out_end])
        # move along one time step
        in_start += 1
    return np.array(X), np.array(y)

In [ ]:
from tqdm import tqdm

def make_predictions(models, X, n_steps, model='all'):
    predictions = []
    # Record the starting time to generate predictions
    predictions_start_time = time.time()

    for i in tqdm(range(len(X)), desc="Progress"):
      if model == 'esn':
          row_forecasts = dirRec_strategy(models, X[i, :], n_steps, model)
      else :
          row_forecasts = dirRec_strategy(models, X[i, :], n_steps)
      predictions.append(row_forecasts)

    # Record the ending time of generating predictions
    predictions_end_time = time.time()
    predictions_elapsed_time = predictions_end_time - predictions_start_time
    return predictions, predictions_elapsed_time

In [ ]:
# Best Params
look_back = 300
num_hidden_layers = 2
learning_rate = 6.95812073050639e-05 # 0.0015250869829489782
batch_size = 30 #44
epochs = 20 #82
num_units_layer_0 = 15
num_units_layer_1 = 32

# ESN Params
n_reservoir = 10
sparsity = 0.5
spectral_radius = 10.0
noise = 0.001
look_back = 44

In [ ]:
def create_dirrec_model(num_units_layer_0, num_units_layer_1, learning_rate, model_type, n_inputs = 10):

    if model_type=='lstm':
      model = Sequential()
      model.add(LSTM(units=num_units_layer_0, return_sequences=True))
      model.add(LSTM(units=num_units_layer_1))
      model.add(Dense(1))
      optimizer = keras.optimizers.Adam(lr=learning_rate)
      model.compile(loss='mean_squared_error', optimizer=optimizer)
    elif model_type=='gru':
      model = Sequential()
      model.add(GRU(units=num_units_layer_0, return_sequences=True))
      model.add(GRU(units=num_units_layer_1))
      model.add(Dense(1))
      optimizer = keras.optimizers.Adam(lr=learning_rate)
      model.compile(loss='mean_squared_error', optimizer=optimizer)
    elif model_type=='rnn':
      model = Sequential()
      model.add(SimpleRNN(units=num_units_layer_0, return_sequences=True))
      model.add(SimpleRNN(units=num_units_layer_1))
      model.add(Dense(1))
      optimizer = keras.optimizers.Adam(lr=learning_rate)
      model.compile(loss='mean_squared_error', optimizer=optimizer)
    elif model_type=='esn':
      model = ESN(n_inputs = n_inputs,
        n_outputs = 1,
        n_reservoir = n_reservoir,
        sparsity=sparsity,
        random_state=1234,
        spectral_radius=spectral_radius,
        noise = noise,
        silent = False,
        teacher_scaling = 10,
      )

    return model

In [ ]:
def DirRecStrategy(trainX, trainY, horizon, epochs=20, batch_size=150, model_type='lstm'):

    models = []
    input_set = []
    total_training_elapsed_time = 0

    for h in tqdm(range(horizon)):
        model = create_dirrec_model(num_units_layer_0, num_units_layer_1, learning_rate, model_type, n_inputs=trainX.shape[1])

        # Record the starting time to training the model
        training_start_time = time.time()

        # Train the model with the current input data and target
        if model_type=='esn':
            model.fit(trainX, trainY[:, h])
        else :
            model.fit(trainX, trainY[:, h], epochs=epochs, batch_size=batch_size, verbose=0)


        # Record the ending time of training the model
        training_end_time = time.time()
        training_elapsed_time = training_end_time - training_start_time
        total_training_elapsed_time += training_elapsed_time

        models.append(model)

        # Update the input set with the current model's prediction
        predictions = model.predict(trainX)
        if model_type=='esn':
          trainX = np.concatenate((trainX, predictions), axis=1)
        else:
          trainX = np.concatenate((trainX, predictions[:, np.newaxis]), axis=-1)

    return models, total_training_elapsed_time

In [ ]:
def predict_with_dirrec_models(models, testX, model_type='all'):
    predictions = []

    # Record the starting time to generate predictions
    predicting_start_time = time.time()

    for h in range(horizon):
        model = models[h]

        # Predict using the current model and test data
        prediction = model.predict(testX)
        print("h = ", h)
        predictions.append(prediction)

        if model_type == 'esn':
          testX = np.concatenate((testX, prediction), axis=1)
        else:
          testX = np.concatenate((testX, prediction[:, np.newaxis]), axis=-1)

    # Record the ending time of generate predictions
    predicting_end_time = time.time()
    predicting_elapsed_time = predicting_end_time - predicting_start_time

    return np.array(predictions).T, predicting_elapsed_time

### LSTM Model

#### Horizon = 2

In [ ]:
horizon = 2
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))


models, training_elapsed_time = DirRecStrategy(trainX, trainY, horizon, epochs, batch_size)

print("LSTM Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

testPredict, predictions_elapsed_time = predict_with_dirrec_models(models, testX)

testPredict = np.array(testPredict)
testPredict = np.squeeze(testPredict)

testRMSE = np.sqrt(mean_squared_error(testY, testPredict))
testMAE = mean_absolute_error(testY, testPredict)

print('LSTM Test RMSE : %.5f' % (testRMSE))
print('LSTM Test MAE : %.5f' % (testMAE))
print("LSTM Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size:  10
17991 2000


  0%|          | 0/2 [00:00<?, ?it/s]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


 50%|█████     | 1/2 [01:03<01:03, 63.53s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 3ms/step


100%|██████████| 2/2 [02:02<00:00, 61.06s/it]

LSTM Model Training Elapsed Time : 115.42446 seconds
63/63 [==============================] - 0s 2ms/step


h =  0
63/63 [==============================] - 0s 2ms/step
h =  1
LSTM Test RMSE : 0.02308
LSTM Test MAE : 0.01858
LSTM Elapsed Time To generate Predictions : 0.58381 seconds


#### Horizon = 4

In [ ]:
horizon = 4
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))


models, training_elapsed_time = DirRecStrategy(trainX, trainY, horizon, epochs, batch_size)

print("LSTM Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

testPredict, predictions_elapsed_time = predict_with_dirrec_models(models, testX)

testPredict = np.array(testPredict)
testPredict = np.squeeze(testPredict)

testRMSE = np.sqrt(mean_squared_error(testY, testPredict))
testMAE = mean_absolute_error(testY, testPredict)

print('LSTM Test RMSE : %.5f' % (testRMSE))
print('LSTM Test MAE : %.5f' % (testMAE))
print("LSTM Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size:  10
17991 2000


  0%|          | 0/4 [00:00<?, ?it/s]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


 25%|██▌       | 1/4 [00:53<02:41, 53.94s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


...

 75%|███████▌  | 3/4 [03:29<01:10, 71.00s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 3s 3ms/step


100%|██████████| 4/4 [04:59<00:00, 74.92s/it]

LSTM Model Training Elapsed Time : 286.14430 seconds
52/63 [=======================>......] - ETA: 0s

63/63 [==============================] - 0s 2ms/step
h =  0
63/63 [==============================] - 0s 2ms/step
h =  1
63/63 [==============================] - 0s 2ms/step
h =  2
63/63 [==============================] - 0s 2ms/step
h =  3
LSTM Test RMSE : 0.02935
LSTM Test MAE : 0.02326
LSTM Elapsed Time To generate Predictions : 1.18311 seconds


#### Horizon = 6

In [ ]:
horizon = 6
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))


models, training_elapsed_time = DirRecStrategy(trainX, trainY, horizon, epochs, batch_size)

print("LSTM Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

testPredict, predictions_elapsed_time = predict_with_dirrec_models(models, testX)

testPredict = np.array(testPredict)
testPredict = np.squeeze(testPredict)

testRMSE = np.sqrt(mean_squared_error(testY, testPredict))
testMAE = mean_absolute_error(testY, testPredict)

print('LSTM Test RMSE : %.5f' % (testRMSE))
print('LSTM Test MAE : %.5f' % (testMAE))
print("LSTM Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size:  10
17991 2000


  0%|          | 0/6 [00:00<?, ?it/s]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


 17%|█▋        | 1/6 [00:57<04:46, 57.35s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


...

 83%|████████▎ | 5/6 [05:22<01:03, 63.84s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


100%|██████████| 6/6 [06:51<00:00, 68.55s/it]

LSTM Model Training Elapsed Time : 391.17303 seconds
42/62 [===================>..........] - ETA: 0s

62/62 [==============================] - 0s 3ms/step
h =  0
62/62 [==============================] - 0s 3ms/step
h =  1
62/62 [==============================] - 0s 3ms/step
h =  2
62/62 [==============================] - 0s 3ms/step
h =  3
62/62 [==============================] - 0s 2ms/step
h =  4
62/62 [==============================] - 0s 2ms/step
h =  5
LSTM Test RMSE : 0.03418
LSTM Test MAE : 0.02675
LSTM Elapsed Time To generate Predictions : 2.10200 seconds


#### Horizon = 8

In [ ]:
horizon = 8
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))


models, training_elapsed_time = DirRecStrategy(trainX, trainY, horizon, epochs, batch_size)

print("LSTM Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

testPredict, predictions_elapsed_time = predict_with_dirrec_models(models, testX)

testPredict = np.array(testPredict)
testPredict = np.squeeze(testPredict)

testRMSE = np.sqrt(mean_squared_error(testY, testPredict))
testMAE = mean_absolute_error(testY, testPredict)

print('LSTM Test RMSE : %.5f' % (testRMSE))
print('LSTM Test MAE : %.5f' % (testMAE))
print("LSTM Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size:  10
17991 2000


  0%|          | 0/8 [00:00<?, ?it/s]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


 12%|█▎        | 1/8 [01:09<08:04, 69.19s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


...

 88%|████████▊ | 7/8 [07:54<01:12, 72.45s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


100%|██████████| 8/8 [08:46<00:00, 65.87s/it]

LSTM Model Training Elapsed Time : 502.98660 seconds
62/62 [==============================] - 0s 2ms/step


h =  0
62/62 [==============================] - 0s 2ms/step
h =  1
62/62 [==============================] - 0s 2ms/step
h =  2
62/62 [==============================] - 0s 2ms/step
h =  3
62/62 [==============================] - 0s 2ms/step
h =  4
62/62 [==============================] - 0s 2ms/step
h =  5
62/62 [==============================] - 0s 2ms/step
h =  6
62/62 [==============================] - 0s 2ms/step
h =  7
LSTM Test RMSE : 0.03734
LSTM Test MAE : 0.02914
LSTM Elapsed Time To generate Predictions : 2.32222 seconds


#### Horizon = 10

In [ ]:
horizon = 10
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))


models, training_elapsed_time = DirRecStrategy(trainX, trainY, horizon, epochs, batch_size)

print("LSTM Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

testPredict, predictions_elapsed_time = predict_with_dirrec_models(models, testX)

testPredict = np.array(testPredict)
testPredict = np.squeeze(testPredict)

testRMSE = np.sqrt(mean_squared_error(testY, testPredict))
testMAE = mean_absolute_error(testY, testPredict)

print('LSTM Test RMSE : %.5f' % (testRMSE))
print('LSTM Test MAE : %.5f' % (testMAE))
print("LSTM Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size:  10
17991 2000


  0%|          | 0/10 [00:00<?, ?it/s]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


 10%|█         | 1/10 [00:52<07:54, 52.75s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


...

 90%|█████████ | 9/10 [09:22<00:59, 59.92s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


100%|██████████| 10/10 [10:50<00:00, 65.09s/it]

LSTM Model Training Elapsed Time : 622.08811 seconds
62/62 [==============================] - 0s 2ms/step


h =  0
62/62 [==============================] - 0s 2ms/step
h =  1
62/62 [==============================] - 0s 2ms/step
h =  2
62/62 [==============================] - 0s 2ms/step
h =  3
62/62 [==============================] - 0s 2ms/step
h =  4
62/62 [==============================] - 0s 2ms/step
h =  5
62/62 [==============================] - 0s 2ms/step
h =  6
62/62 [==============================] - 0s 2ms/step
h =  7
62/62 [==============================] - 0s 2ms/step
h =  8
62/62 [==============================] - 0s 2ms/step
h =  9
LSTM Test RMSE : 0.04090
LSTM Test MAE : 0.03205
LSTM Elapsed Time To generate Predictions : 2.90129 seconds


### GRU Model

#### Horizon = 2

In [ ]:
horizon = 2
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

models, training_elapsed_time = DirRecStrategy(trainX, trainY, horizon, epochs, batch_size, model_type='gru')

print("GRU Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

testPredict, predictions_elapsed_time = predict_with_dirrec_models(models, testX)

testPredict = np.array(testPredict)
testPredict = np.squeeze(testPredict)

testRMSE = np.sqrt(mean_squared_error(testY, testPredict))
testMAE = mean_absolute_error(testY, testPredict)

print('GRU Test RMSE : %.5f' % (testRMSE))
print('GRU Test MAE : %.5f' % (testMAE))
print("GRU Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size:  10
17991 2000


  0%|          | 0/2 [00:00<?, ?it/s]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


 50%|█████     | 1/2 [01:29<01:29, 89.60s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


100%|██████████| 2/2 [02:22<00:00, 71.10s/it]

GRU Model Training Elapsed Time : 136.87591 seconds
63/63 [==============================] - 0s 2ms/step


h =  0
63/63 [==============================] - 0s 2ms/step
h =  1
GRU Test RMSE : 0.02330
GRU Test MAE : 0.01870
GRU Elapsed Time To generate Predictions : 0.59887 seconds


#### Horizon = 4

In [ ]:
horizon = 4
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

models, training_elapsed_time = DirRecStrategy(trainX, trainY, horizon, epochs, batch_size, model_type='gru')

print("GRU Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

testPredict, predictions_elapsed_time = predict_with_dirrec_models(models, testX)

testPredict = np.array(testPredict)
testPredict = np.squeeze(testPredict)

testRMSE = np.sqrt(mean_squared_error(testY, testPredict))
testMAE = mean_absolute_error(testY, testPredict)

print('GRU Test RMSE : %.5f' % (testRMSE))
print('GRU Test MAE : %.5f' % (testMAE))
print("GRU Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size:  10
17991 2000


  0%|          | 0/4 [00:00<?, ?it/s]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


 25%|██▌       | 1/4 [01:07<03:22, 67.56s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 3s 3ms/step


...

 75%|███████▌  | 3/4 [03:18<01:06, 66.03s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


100%|██████████| 4/4 [04:23<00:00, 65.92s/it]

GRU Model Training Elapsed Time : 253.00411 seconds
50/63 [======================>.......] - ETA: 0s

63/63 [==============================] - 0s 2ms/step
h =  0
63/63 [==============================] - 0s 2ms/step
h =  1
63/63 [==============================] - 0s 2ms/step
h =  2
63/63 [==============================] - 0s 2ms/step
h =  3
GRU Test RMSE : 0.02944
GRU Test MAE : 0.02329
GRU Elapsed Time To generate Predictions : 0.91815 seconds


#### Horizon = 6

In [ ]:
horizon = 6
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

models, training_elapsed_time = DirRecStrategy(trainX, trainY, horizon, epochs, batch_size, model_type='gru')

print("GRU Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

testPredict, predictions_elapsed_time = predict_with_dirrec_models(models, testX)

testPredict = np.array(testPredict)
testPredict = np.squeeze(testPredict)

testRMSE = np.sqrt(mean_squared_error(testY, testPredict))
testMAE = mean_absolute_error(testY, testPredict)

print('GRU Test RMSE : %.5f' % (testRMSE))
print('GRU Test MAE : %.5f' % (testMAE))
print("GRU Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size:  10
17991 2000


  0%|          | 0/6 [00:00<?, ?it/s]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


 17%|█▋        | 1/6 [01:28<07:22, 88.43s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 4s 7ms/step


...

 83%|████████▎ | 5/6 [06:14<01:16, 76.20s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 3s 2ms/step


100%|██████████| 6/6 [07:45<00:00, 77.60s/it]

GRU Model Training Elapsed Time : 445.41206 seconds
49/62 [======================>.......] - ETA: 0s

62/62 [==============================] - 0s 2ms/step
h =  0
62/62 [==============================] - 0s 2ms/step
h =  1
62/62 [==============================] - 0s 2ms/step
h =  2
62/62 [==============================] - 0s 3ms/step
h =  3
62/62 [==============================] - 0s 3ms/step
h =  4
62/62 [==============================] - 0s 3ms/step
h =  5
GRU Test RMSE : 0.03421
GRU Test MAE : 0.02679
GRU Elapsed Time To generate Predictions : 2.27884 seconds


#### Horizon = 8

In [ ]:
horizon = 8
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

models, training_elapsed_time = DirRecStrategy(trainX, trainY, horizon, epochs, batch_size, model_type='gru')

print("GRU Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

testPredict, predictions_elapsed_time = predict_with_dirrec_models(models, testX)

testPredict = np.array(testPredict)
testPredict = np.squeeze(testPredict)

testRMSE = np.sqrt(mean_squared_error(testY, testPredict))
testMAE = mean_absolute_error(testY, testPredict)

print('GRU Test RMSE : %.5f' % (testRMSE))
print('GRU Test MAE : %.5f' % (testMAE))
print("GRU Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size:  10
17991 2000


  0%|          | 0/8 [00:00<?, ?it/s]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


 12%|█▎        | 1/8 [01:00<07:03, 60.48s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


...

 88%|████████▊ | 7/8 [08:06<01:06, 66.72s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


100%|██████████| 8/8 [09:00<00:00, 67.55s/it]

GRU Model Training Elapsed Time : 516.29333 seconds
62/62 [==============================] - 0s 2ms/step


h =  0
62/62 [==============================] - 0s 2ms/step
h =  1
62/62 [==============================] - 0s 2ms/step
h =  2
62/62 [==============================] - 0s 2ms/step
h =  3
62/62 [==============================] - 0s 3ms/step
h =  4
62/62 [==============================] - 0s 3ms/step
h =  5
62/62 [==============================] - 0s 3ms/step
h =  6
62/62 [==============================] - 0s 3ms/step
h =  7
GRU Test RMSE : 0.03723
GRU Test MAE : 0.02905
GRU Elapsed Time To generate Predictions : 2.68460 seconds


#### Horizon = 10

In [ ]:
horizon = 10
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

models, training_elapsed_time = DirRecStrategy(trainX, trainY, horizon, epochs, batch_size, model_type='gru')

print("GRU Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

testPredict, predictions_elapsed_time = predict_with_dirrec_models(models, testX)

testPredict = np.array(testPredict)
testPredict = np.squeeze(testPredict)

testRMSE = np.sqrt(mean_squared_error(testY, testPredict))
testMAE = mean_absolute_error(testY, testPredict)

print('GRU Test RMSE : %.5f' % (testRMSE))
print('GRU Test MAE : %.5f' % (testMAE))
print("GRU Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size:  10
17991 2000


  0%|          | 0/10 [00:00<?, ?it/s]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


 10%|█         | 1/10 [00:53<08:01, 53.48s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


...

 90%|█████████ | 9/10 [09:06<00:59, 59.21s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


100%|██████████| 10/10 [10:01<00:00, 60.17s/it]

GRU Model Training Elapsed Time : 572.62931 seconds
62/62 [==============================] - 0s 2ms/step


h =  0
62/62 [==============================] - 0s 2ms/step
h =  1
62/62 [==============================] - 0s 2ms/step
h =  2
62/62 [==============================] - 0s 2ms/step
h =  3
62/62 [==============================] - 0s 2ms/step
h =  4
62/62 [==============================] - 0s 2ms/step
h =  5
62/62 [==============================] - 0s 2ms/step
h =  6
62/62 [==============================] - 0s 3ms/step
h =  7
62/62 [==============================] - 0s 2ms/step
h =  8
62/62 [==============================] - 0s 2ms/step
h =  9
GRU Test RMSE : 0.04087
GRU Test MAE : 0.03203
GRU Elapsed Time To generate Predictions : 2.45680 seconds


### RNN Model

#### Horizon = 2

In [ ]:
horizon = 2
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

models, training_elapsed_time = DirRecStrategy(trainX, trainY, horizon, epochs, batch_size, model_type='rnn')

print("RNN Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

testPredict, predictions_elapsed_time = predict_with_dirrec_models(models, testX)

testPredict = np.array(testPredict)
testPredict = np.squeeze(testPredict)

testRMSE = np.sqrt(mean_squared_error(testY, testPredict))
testMAE = mean_absolute_error(testY, testPredict)

print('RNN Test RMSE : %.5f' % (testRMSE))
print('RNN Test MAE : %.5f' % (testMAE))
print("RNN Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size:  10
17991 2000


  0%|          | 0/2 [00:00<?, ?it/s]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 1s 2ms/step


 50%|█████     | 1/2 [00:30<00:30, 30.70s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 1s 2ms/step


100%|██████████| 2/2 [01:15<00:00, 37.59s/it]

RNN Model Training Elapsed Time : 72.18820 seconds
63/63 [==============================] - 0s 2ms/step


h =  0
63/63 [==============================] - 0s 2ms/step
h =  1
RNN Test RMSE : 0.02280
RNN Test MAE : 0.01835
RNN Elapsed Time To generate Predictions : 0.39885 seconds


#### Horizon = 4

In [ ]:
horizon = 4
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

models, training_elapsed_time = DirRecStrategy(trainX, trainY, horizon, epochs, batch_size, model_type='rnn')

print("RNN Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

testPredict, predictions_elapsed_time = predict_with_dirrec_models(models, testX)

testPredict = np.array(testPredict)
testPredict = np.squeeze(testPredict)

testRMSE = np.sqrt(mean_squared_error(testY, testPredict))
testMAE = mean_absolute_error(testY, testPredict)

print('RNN Test RMSE : %.5f' % (testRMSE))
print('RNN Test MAE : %.5f' % (testMAE))
print("RNN Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size:  10
17991 2000


  0%|          | 0/4 [00:00<?, ?it/s]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 1s 2ms/step


 25%|██▌       | 1/4 [00:31<01:34, 31.39s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 1s 2ms/step


...

 75%|███████▌  | 3/4 [02:00<00:41, 41.56s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 1s 2ms/step


100%|██████████| 4/4 [02:36<00:00, 39.10s/it]

RNN Model Training Elapsed Time : 150.03004 seconds
54/63 [========================>.....] - ETA: 0s

63/63 [==============================] - 0s 2ms/step
h =  0
63/63 [==============================] - 0s 2ms/step
h =  1
63/63 [==============================] - 0s 2ms/step
h =  2
63/63 [==============================] - 0s 2ms/step
h =  3
RNN Test RMSE : 0.02917
RNN Test MAE : 0.02308
RNN Elapsed Time To generate Predictions : 0.86991 seconds


#### Horizon = 6

In [ ]:
horizon = 6
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

models, training_elapsed_time = DirRecStrategy(trainX, trainY, horizon, epochs, batch_size, model_type='rnn')

print("RNN Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

testPredict, predictions_elapsed_time = predict_with_dirrec_models(models, testX)

testPredict = np.array(testPredict)
testPredict = np.squeeze(testPredict)

testRMSE = np.sqrt(mean_squared_error(testY, testPredict))
testMAE = mean_absolute_error(testY, testPredict)

print('RNN Test RMSE : %.5f' % (testRMSE))
print('RNN Test MAE : %.5f' % (testMAE))
print("RNN Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size:  10
17991 2000


  0%|          | 0/6 [00:00<?, ?it/s]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 1s 2ms/step


 17%|█▋        | 1/6 [00:36<03:02, 36.59s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 1s 2ms/step


...

 83%|████████▎ | 5/6 [03:15<00:40, 40.22s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


100%|██████████| 6/6 [04:01<00:00, 40.32s/it]

RNN Model Training Elapsed Time : 231.09158 seconds
62/62 [==============================] - 0s 2ms/step


h =  0
62/62 [==============================] - 0s 2ms/step
h =  1
62/62 [==============================] - 0s 2ms/step
h =  2
62/62 [==============================] - 0s 2ms/step
h =  3
62/62 [==============================] - 0s 2ms/step
h =  4
62/62 [==============================] - 0s 2ms/step
h =  5
RNN Test RMSE : 0.03445
RNN Test MAE : 0.02697
RNN Elapsed Time To generate Predictions : 1.38626 seconds


#### Horizon = 8

In [ ]:
horizon = 8
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

models, training_elapsed_time = DirRecStrategy(trainX, trainY, horizon, epochs, batch_size, model_type='rnn')

print("RNN Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

testPredict, predictions_elapsed_time = predict_with_dirrec_models(models, testX)

testPredict = np.array(testPredict)
testPredict = np.squeeze(testPredict)

testRMSE = np.sqrt(mean_squared_error(testY, testPredict))
testMAE = mean_absolute_error(testY, testPredict)

print('RNN Test RMSE : %.5f' % (testRMSE))
print('RNN Test MAE : %.5f' % (testMAE))
print("RNN Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size:  10
17991 2000


  0%|          | 0/8 [00:00<?, ?it/s]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 1s 2ms/step


 12%|█▎        | 1/8 [00:44<05:13, 44.73s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 3ms/step


...

562/562 [==============================] - 1s 2ms/step


 88%|████████▊ | 7/8 [04:32<00:37, 37.15s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 1s 2ms/step


100%|██████████| 8/8 [05:06<00:00, 38.33s/it]


RNN Model Training Elapsed Time : 293.66448 seconds
62/62 [==============================] - 0s 3ms/step
h =  0
62/62 [==============================] - 0s 3ms/step
h =  1
62/62 [==============================] - 0s 2ms/step
h =  2
62/62 [==============================] - 0s 2ms/step
h =  3
62/62 [==============================] - 0s 2ms/step
h =  4
62/62 [==============================] - 0s 2ms/step
h =  5
62/62 [==============================] - 0s 2ms/step
h =  6
62/62 [==============================] - 0s 2ms/step
h =  7
RNN Test RMSE : 0.03718
RNN Test MAE : 0.02899
RNN Elapsed Time To generate Predictions : 5.07520 seconds


#### Horizon = 10

In [ ]:
horizon = 10
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

models, training_elapsed_time = DirRecStrategy(trainX, trainY, horizon, epochs, batch_size, model_type='rnn')

print("RNN Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

testPredict, predictions_elapsed_time = predict_with_dirrec_models(models, testX)

testPredict = np.array(testPredict)
testPredict = np.squeeze(testPredict)

testRMSE = np.sqrt(mean_squared_error(testY, testPredict))
testMAE = mean_absolute_error(testY, testPredict)

print('RNN Test RMSE : %.5f' % (testRMSE))
print('RNN Test MAE : %.5f' % (testMAE))
print("RNN Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size:  10
17991 2000


  0%|          | 0/10 [00:00<?, ?it/s]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 1s 2ms/step


 10%|█         | 1/10 [00:38<05:45, 38.43s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 1s 2ms/step


...

 90%|█████████ | 9/10 [05:35<00:38, 38.39s/it]WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.


562/562 [==============================] - 2s 2ms/step


100%|██████████| 10/10 [06:11<00:00, 37.16s/it]

RNN Model Training Elapsed Time : 352.98019 seconds
62/62 [==============================] - 0s 2ms/step


h =  0
62/62 [==============================] - 0s 2ms/step
h =  1
62/62 [==============================] - 0s 2ms/step
h =  2
62/62 [==============================] - 0s 2ms/step
h =  3
62/62 [==============================] - 0s 2ms/step
h =  4
62/62 [==============================] - 0s 2ms/step
h =  5
62/62 [==============================] - 0s 2ms/step
h =  6
62/62 [==============================] - 0s 2ms/step
h =  7
62/62 [==============================] - 0s 2ms/step
h =  8
62/62 [==============================] - 0s 2ms/step
h =  9
RNN Test RMSE : 0.04159
RNN Test MAE : 0.03256
RNN Elapsed Time To generate Predictions : 2.54613 seconds


#### ESN Model

#### Horizon = 2

In [ ]:
horizon = 2
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

models, training_elapsed_time = DirRecStrategy(trainX, trainY, horizon=horizon, model_type='esn')

print("ESN Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

testPredict, predictions_elapsed_time = predict_with_dirrec_models(models, testX, 'esn')

testPredict = np.array(testPredict)
testPredict = np.squeeze(testPredict)

testRMSE = np.sqrt(mean_squared_error(testY, testPredict))
testMAE = mean_absolute_error(testY, testPredict)

print('ESN Test RMSE : %.5f' % (testRMSE))
print('ESN Test MAE : %.5f' % (testMAE))
print("ESN Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size:  10
17991 2000


  0%|          | 0/2 [00:00<?, ?it/s]

harvesting states...
fitting...
training error:
0.01655028175313723


 50%|█████     | 1/2 [00:00<00:00,  1.73it/s]

harvesting states...
fitting...
training error:
0.023382400974606313


100%|██████████| 2/2 [00:01<00:00,  1.81it/s]

ESN Model Training Elapsed Time : 0.50169 seconds
h =  0
h =  1
ESN Test RMSE : 0.02087
ESN Test MAE : 0.01690
ESN Elapsed Time To generate Predictions : 0.05832 seconds


#### Horizon = 4

In [ ]:
horizon = 4
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

models, training_elapsed_time = DirRecStrategy(trainX, trainY, horizon=horizon, model_type='esn')

print("ESN Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

testPredict, predictions_elapsed_time = predict_with_dirrec_models(models, testX, 'esn')

testPredict = np.array(testPredict)
testPredict = np.squeeze(testPredict)

testRMSE = np.sqrt(mean_squared_error(testY, testPredict))
testMAE = mean_absolute_error(testY, testPredict)

print('ESN Test RMSE : %.5f' % (testRMSE))
print('ESN Test MAE : %.5f' % (testMAE))
print("ESN Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size:  10
17991 2000


  0%|          | 0/4 [00:00<?, ?it/s]

harvesting states...
fitting...
training error:
0.016549919257358093


 25%|██▌       | 1/4 [00:00<00:01,  1.91it/s]

harvesting states...
fitting...
training error:
0.0233836033035889


...

harvesting states...
fitting...
training error:
0.02858703921914926


 75%|███████▌  | 3/4 [00:01<00:00,  1.88it/s]

harvesting states...
fitting...
training error:
0.0287247667685399


100%|██████████| 4/4 [00:02<00:00,  1.89it/s]

ESN Model Training Elapsed Time : 0.95320 seconds
h =  0
h =  1
h =  2
h =  3
ESN Test RMSE : 0.02735
ESN Test MAE : 0.02157
ESN Elapsed Time To generate Predictions : 0.11133 seconds


#### Horizon = 6

In [ ]:
horizon = 6
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

models, training_elapsed_time = DirRecStrategy(trainX, trainY, horizon=horizon, model_type='esn')

print("ESN Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

testPredict, predictions_elapsed_time = predict_with_dirrec_models(models, testX, 'esn')

testPredict = np.array(testPredict)
testPredict = np.squeeze(testPredict)

testRMSE = np.sqrt(mean_squared_error(testY, testPredict))
testMAE = mean_absolute_error(testY, testPredict)

print('ESN Test RMSE : %.5f' % (testRMSE))
print('ESN Test MAE : %.5f' % (testMAE))
print("ESN Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size:  10
17991 2000


  0%|          | 0/6 [00:00<?, ?it/s]

harvesting states...
fitting...
training error:
0.0165506154101317


 17%|█▋        | 1/6 [00:00<00:04,  1.13it/s]

harvesting states...
fitting...
training error:
0.023376865982590894


...

harvesting states...
fitting...
training error:
0.03629763451010729


 83%|████████▎ | 5/6 [00:03<00:00,  1.75it/s]

harvesting states...
fitting...
training error:
0.03890221961250927


100%|██████████| 6/6 [00:03<00:00,  1.66it/s]

ESN Model Training Elapsed Time : 1.65765 seconds
h =  0
h =  1
h =  2
h =  3
h =  4
h =  5
ESN Test RMSE : 0.03199
ESN Test MAE : 0.02493
ESN Elapsed Time To generate Predictions : 0.16441 seconds


#### Horizon = 8

In [ ]:
horizon = 8
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

models, training_elapsed_time = DirRecStrategy(trainX, trainY, horizon=horizon, model_type='esn')

print("ESN Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

testPredict, predictions_elapsed_time = predict_with_dirrec_models(models, testX, 'esn')

testPredict = np.array(testPredict)
testPredict = np.squeeze(testPredict)

testRMSE = np.sqrt(mean_squared_error(testY, testPredict))
testMAE = mean_absolute_error(testY, testPredict)

print('ESN Test RMSE : %.5f' % (testRMSE))
print('ESN Test MAE : %.5f' % (testMAE))
print("ESN Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size:  10
17991 2000


  0%|          | 0/8 [00:00<?, ?it/s]

harvesting states...
fitting...
training error:
0.01655073896057443


 12%|█▎        | 1/8 [00:00<00:03,  1.84it/s]

harvesting states...
fitting...
training error:
0.023385260565537382


...

harvesting states...
fitting...
training error:
0.03827139022815909


 88%|████████▊ | 7/8 [00:03<00:00,  1.87it/s]

harvesting states...
fitting...
training error:
0.04492696730294201


100%|██████████| 8/8 [00:04<00:00,  1.85it/s]


ESN Model Training Elapsed Time : 1.95928 seconds
h =  0
h =  1
h =  2
h =  3
h =  4
h =  5
h =  6
h =  7
ESN Test RMSE : 0.03670
ESN Test MAE : 0.02833
ESN Elapsed Time To generate Predictions : 0.22359 seconds


#### Horizon = 10

In [ ]:
horizon = 10
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))

# reshape into X=t and Y=t+1
trainX, trainY = create_multistep_dataset(train, look_back, horizon)
testX, testY = create_multistep_dataset(test, look_back, horizon)

models, training_elapsed_time = DirRecStrategy(trainX, trainY, horizon=horizon, model_type='esn')

print("ESN Model Training Elapsed Time : %.5f" % (training_elapsed_time), "seconds")

testPredict, predictions_elapsed_time = predict_with_dirrec_models(models, testX, 'esn')

testPredict = np.array(testPredict)
testPredict = np.squeeze(testPredict)

testRMSE = np.sqrt(mean_squared_error(testY, testPredict))
testMAE = mean_absolute_error(testY, testPredict)

print('ESN Test RMSE : %.5f' % (testRMSE))
print('ESN Test MAE : %.5f' % (testMAE))
print("ESN Elapsed Time To generate Predictions : %.5f" % (predictions_elapsed_time), "seconds")

Window Size:  10
17991 2000


  0%|          | 0/10 [00:00<?, ?it/s]

harvesting states...
fitting...
training error:
0.016550036247321924


 10%|█         | 1/10 [00:00<00:04,  1.88it/s]

harvesting states...
fitting...
training error:
0.023371042046000434


...

harvesting states...
fitting...
training error:
0.04515503103011756


 90%|█████████ | 9/10 [00:05<00:00,  1.59it/s]

harvesting states...
fitting...
training error:
0.04789086195047638


100%|██████████| 10/10 [00:06<00:00,  1.65it/s]


ESN Model Training Elapsed Time : 2.70162 seconds
h =  0
h =  1
h =  2
h =  3
h =  4
h =  5
h =  6
h =  7
h =  8
h =  9
ESN Test RMSE : 0.04095
ESN Test MAE : 0.03150
ESN Elapsed Time To generate Predictions : 0.28032 seconds


In [ ]:
0.04090-0.04095

-5.000000000000143e-05

## Finding best ESN params

In [ ]:
!pip install -q optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.8 MB/s eta 0:00:00


In [ ]:
import optuna
from optuna.samplers import TPESampler

In [ ]:
def create_dirrec_esn_model(n_inputs, n_reservoir, sparsity, spectral_radius, noise):

    model = ESN(n_inputs = n_inputs,
      n_outputs = 1,
      n_reservoir = n_reservoir,
      sparsity=sparsity,
      random_state=1234,
      spectral_radius=spectral_radius,
      noise = noise,
      silent = False,
      teacher_scaling = 10,
    )

    return model

In [ ]:
def DirRecESNStrategy(trainX, trainY, horizon, n_reservoir, sparsity, spectral_radius, noise):

    models = []
    input_set = []
    total_training_elapsed_time = 0

    for h in tqdm(range(horizon)):
        model = create_dirrec_esn_model(trainX.shape[1], n_reservoir, sparsity, spectral_radius, noise)

        # Record the starting time to training the model
        training_start_time = time.time()

        # Train the model with the current input data and target
        model.fit(trainX, trainY[:, h])


        # Record the ending time of training the model
        training_end_time = time.time()
        training_elapsed_time = training_end_time - training_start_time
        total_training_elapsed_time += training_elapsed_time

        models.append(model)

        # Update the input set with the current model's prediction
        predictions = model.predict(trainX)
        trainX = np.concatenate((trainX, predictions), axis=1)

    return models, total_training_elapsed_time

In [ ]:
def predict_with_dirrec_esn_models(models, testX, model_type='all'):
    predictions = []

    # Record the starting time to generate predictions
    predicting_start_time = time.time()

    for h in range(horizon):
        model = models[h]

        # Predict using the current model and test data
        prediction = model.predict(testX)
        print("h = ", h)
        predictions.append(prediction)

        if model_type == 'esn':
          testX = np.concatenate((testX, prediction), axis=1)
        else:
          testX = np.concatenate((testX, prediction[:, np.newaxis]), axis=-1)

    # Record the ending time of generate predictions
    predicting_end_time = time.time()
    predicting_elapsed_time = predicting_end_time - predicting_start_time

    return np.array(predictions).T, predicting_elapsed_time

In [ ]:
horizon = 8
node1_delay = df[['node1_delay']]
node1_delay_dataset = node1_delay.values
window_size, slide_size = smooth_ASAP(node1_delay_dataset, resolution=50)
print("Window Size: ", window_size)

denoised_node1_delay_dataset = moving_average(node1_delay_dataset, window_size)

# split into train and test sets
train_size = int(len(denoised_node1_delay_dataset) * 0.9)
test_size = len(denoised_node1_delay_dataset) - train_size
train, test = denoised_node1_delay_dataset[0:train_size], denoised_node1_delay_dataset[train_size:]

print(len(train), len(test))


# Define the objective function for Optuna optimization
def objective(trial):
    # Sample hyperparameters to search

    n_reservoir = trial.suggest_int('n_reservoir', 10, 1000)   # -
    sparsity = trial.suggest_categorical('sparsity', [0.01, 0.1, 0.2, 0.3, 0.4, 0.5])   # -
    spectral_radius = trial.suggest_categorical('spectral_radius', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.25, 10.0])   # - spectral radius of W
    noise = trial.suggest_categorical('noise', [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009])       # - Noise Set
    look_back = trial.suggest_int('look_back', 10, 500)

    # reshape into X=t and Y=t+1
    trainX, trainY = create_multistep_dataset(train, look_back, horizon)
    validX, validY = create_multistep_dataset(test, look_back, horizon)

    models, training_elapsed_time = DirRecESNStrategy(trainX, trainY, horizon, n_reservoir, sparsity, spectral_radius, noise)

    # make predictions
    testPredict, predictions_elapsed_time = predict_with_dirrec_esn_models(models, validX, 'esn')
    testPredict = np.array(testPredict)
    testPredict = np.squeeze(testPredict)

    testRMSE = np.sqrt(mean_squared_error(validY, testPredict))

    return testRMSE

# Create the Optuna study
study = optuna.create_study(direction='minimize')

# Record the starting time to generate predictions
start_time = time.time()

# Run the optimization
study.optimize(objective, n_trials=100)

# Record the ending time
end_time = time.time()
elapsed_time = end_time - start_time

print('done')
print("ESN HyperParameters Tuning Elapsed Time : %.5f" % (elapsed_time), "seconds")

# Print the best parameters and corresponding loss
best_params = study.best_params
best_loss = study.best_value
print("Best Parameters: ", best_params)
print("Best Loss: ", best_loss)

[I 2023-09-10 00:12:42,605] A new study created in memory with name: no-name-32d17725-11cd-408e-b1e6-94d41a523d6a


Window Size:  10
17991 2000


  0%|          | 0/8 [00:00<?, ?it/s]

harvesting states...
fitting...
training error:
0.01596023157963312


 12%|█▎        | 1/8 [00:22<02:39, 22.74s/it]

...

harvesting states...
fitting...
training error:
0.042543674830532986


100%|██████████| 8/8 [00:22<00:00,  2.85s/it]


h =  0
h =  1
h =  2
h =  3
h =  4
h =  5
h =  

[I 2023-09-10 03:04:43,281] Trial 99 finished with value: 0.03455353675466214 and parameters: {'n_reservoir': 10, 'sparsity': 0.5, 'spectral_radius': 10.0, 'noise': 0.001, 'look_back': 441}. Best is trial 97 with value: 0.03445824494068749.


6
h =  7
done
ESN HyperParameters Tuning Elapsed Time : 10320.67854 seconds
Best Parameters:  {'n_reservoir': 10, 'sparsity': 0.5, 'spectral_radius': 10.0, 'noise': 0.001, 'look_back': 444}
Best Loss:  0.03445824494068749
